# 量子计算
$$
\newcommand{\braket}[2]{\left.\left\langle#1\right|#2\right\rangle}
\newcommand{\sbraket}[1]{\braket{#1}{#1}}
\newcommand{\bracket}[3]{\left.\langle#1\right.\left|#2\right|\left.#3\right\rangle}
\newcommand{\bra}[1]{\left\langle#1\right|}
\newcommand{\ket}[1]{\left|#1\right\rangle}
\newcommand{\norm}[1]{\left\lVert#1\rVert\right}
\newcommand{\exp}[1]{\left\langle#1\right\rangle}
$$

本篇是 *UIUC PHYS 370 Introduction to Quantum Information and Computing* 的学习笔记，涵盖了量子电路、量子算法等话题。

In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

## 目录

1. [线性代数](#linear-algebra)
    - [矢量与狄拉克记号](#vectors-and-dirac-notation)
    - [线性变换与矩阵](#linear-transformation-and-matrices)
    - [内积空间](#inner-product-space)
    - [矩阵的性质](#matrix-properties)
    - [特征值与特征矢量](#eigenvalues-and-eigenvectors)
    - [正规矩阵](#normal-matrices)
    - [张量积](#tensor-product)
    
2. [量子力学简介](#introductory-quantum-mechanics)
    - [波函数与算符](#wave-functions-and-operators)
    - [希尔伯特空间](#hilbert-space)
    - [量子力学的统计解释](#statistical-interpretation)
    - [三维空间与多粒子系统](#3d-space-and-multiparticle-systems)
    - [对称性](#symmetry)
    - [自旋](#spin)
    
3. [量子电路](#quantum-circuits)

## 线性代数 <a name="linear-algebra"></a>

### 矢量与狄拉克记号 <a name="vectors-and-dirac-notation"></a>

量子力学建立于线性代数之上；本篇笔记中我们会大量使用线性代数中的概念，如矢量、矩阵、特征值等；它们在量子力学中具有特定的物理意义。由于物理学家采用了有别于数学家的记号，即使是对线性代数非常了解的同学也推荐快速浏览一下本章节。

**矢量（Vector）** 是 **矢量空间（Vector Space）** 中的量。从初等的角度可以将它们理解成带有方向的量，或空间中的一个点，与仅有大小的 **标量（Scalar）** 相对。从抽象的角度，则定义矢量空间为定义了矢量加法：$+: V\times V \to V$ 和标量乘法 $\cdot: V\times F \to V$ 的集合 $V(F)$，且满足：

- 标量集合 $F$ 是一个数域，即定义了封闭的加减乘除运算。本篇笔记中出现的矢量空间一律 $F=\mathbb{C}$。
- 矢量加法和标量乘法均封闭。
- 矢量加法满足交换律、结合律，并存在零矢量 $0$ 使得 $v + 0 = 0 + v = v$ 对于任意 $v \in V$ 成立。
- 标量乘法满足交换律、结合律、和分配律：$a(u + v) = au + av$ 且 $(a + b)v = av + bv$ 对于任意 $a, b \in F, u, v \in V$ 成立。

数学中通常不对矢量进行任何特殊记号，仅使用 $u, v, w$ 等末位拉丁字母与标量 $a, b, c$ 之类的分开；有时也会用上面带箭头的形式（如 $\vec{u}$）。物理中则会将矢量符号加粗，写成 $\mathbf{a}, \mathbf{b}$ 等样式。量子力学中使用的是 **狄拉克记号（Dirac Notation）**，或 **左右矢记号（Bra-ket Notation）**，将矢量记作 $|\psi\rangle$ 这样希腊字母套在 $|\rangle$ 符号中的形式。

值得特别一提的是，虽然在记号 $\ket{\alpha}$ 中，$\alpha$ 只是一个标签，但是在实践中它也可以表示矢量 $\ket{\alpha}$ 本身。比如笔记中可能会出现类如 $\ket{k\alpha}$ 的简记法，此时它的含义是 $k\ket{\alpha}$，而 $k\alpha$ 是这个矢量的标签。

一集矢量 $|\alpha\rangle, |\beta\rangle, |\gamma\rangle$ 称为 **线性无关的（Linearly Independent）**，如果它们的 **线性组合（Linear Combination）**（下面方程等号左侧的形式）满足：

$$
\begin{equation*}
    a|\alpha\rangle + b|\beta\rangle + c|\gamma\rangle + \dots = 0
\end{equation*}
$$

当且仅当 $a = b = c = \dots = 0$ 时成立。换句话说，这集矢量中的任一个都不能表示为其它矢量的加权和。一集矢量 **张成（Span）** 的空间定义为其所有线性组合形成的矢量空间。如果一个矢量空间 $V$ 可以被一集线性无关的矢量张成，这集向量就是这个 $V$ 的一个 **基（Basis）**，此时可以将矢量空间中任一个元素 $\ket{\alpha}$ 分解为：

$$
\begin{equation*}
    |\alpha\rangle = a_1|e_1\rangle + \dots + a_n|e_n\rangle
\end{equation*}
$$

我们发现，如果给定了矢量空间的一个基，那么任意矢量都可以通过 $n$ 个标量唯一地表示，这样就得到了矢量的另一种形式：

$$
\begin{equation*}
    |\alpha\rangle = \begin{pmatrix} a_1 \\ \vdots \\ a_n \end{pmatrix}
\end{equation*}
$$

不难从矢量空间运算的特点得到下面的结论：

$$
\begin{equation*}
    |\alpha\rangle + |\beta\rangle
        = \begin{pmatrix} a_1 + b_1 \\ \vdots \\ a_n + b_n \end{pmatrix} \qquad
    \lambda|\alpha\rangle
        = \begin{pmatrix} ca_1 \\ \vdots \\ ca_n \end{pmatrix} \qquad
    |-\alpha\rangle
        = \begin{pmatrix} -a_1 \\ \vdots \\ -a_n \end{pmatrix} \qquad
    |0\rangle
        = \begin{pmatrix} 0 \\ \vdots \\ 0 \end{pmatrix}
\end{equation*}
$$

如果矢量空间被大小为 $n$ 的基张成，则称该矢量空间的 **维度（Dimension）** 为 $n$，记作 $\text{dim}(V) = n$。可以看到这和我们在矢量分析中用到的术语含义相似。值得一提的是，对于同一个矢量空间，无论选取的基是什么，其数量总是等于矢量空间的维度；如果一集线性无关的矢量张成了一个矢量空间 $V$，则这集矢量就是 $V$ 的一个基。

至此我们介绍的矢量称为 **右矢（Ket）**，它也是和数学中经典的矢量对应的概念。下面我们将补全另一半，称为 **左矢（Bra）**。从名字中可以看出，这也是一个矢量，因此可以通过某个基用一组数唯一地表示；不过左矢的形式和右矢有所不同：

$$
\begin{equation*}
    \langle\alpha| = \begin{pmatrix} a_1^* & \dots & a_n^* \end{pmatrix}
\end{equation*}
$$

其中 $a_j^*$ 是 $a_j$ 的 **复数共轭（Complex Conjugate）**，定义为 $z^* = \frac{|z|^2}{z}$（$0$ 的共轭则为 $0$）。可以看到，左矢和右矢的形式有微妙的不同，比起后者写成一列数的形式（因此也称为 **列矢量（Column Vector）**），前者将数写在一行上（故也称为 **行矢量（Row Vector）**）。左矢和右矢不在同一个矢量空间，因此两者不能相加；不过两者所在的矢量空间互为对方的 **对偶空间（Dual Space）**，这种特殊的关系使得它们之间可以进行特别的操作，我们会在后续小节中详细说明。

和右矢类似地，左矢的标签可以代表右矢本身（注意不是左矢本身！），如 $\bra{k\alpha}$ 代表了 $k\ket{\alpha}$ 对应的左矢。

Python 中的 `numpy.array` 提供了类似与矢量的数据结构，我们本篇中将使用该类型模拟矢量：

In [3]:
def bra(*args):
    return np.array([args])

def ket(*args):
    res = []
    for arg in args:
        res.append([arg])
    return np.array(res)

def is_bra(v):
    return v.shape[0] == 1

def is_ket(v):
    return v.shape[1] == 1

# dagger 函数将 bra 和 ket 互相转换。后面我们会看到，这不仅适用于矢量
def dagger(a):
    return a.conj().T

print(bra(1+2j, 1j))
print(ket(1+2j, 1j))
print(dagger(bra(1+2j, 1j)))
print(dagger(ket(1+2j, 1j)))

[[1.+2.j 0.+1.j]]
[[1.+2.j]
 [0.+1.j]]
[[1.-2.j]
 [0.-1.j]]
[[1.-2.j 0.-1.j]]


### 线性变换和矩阵

**线性变换（Linear Transformation）** 是从一个矢量空间映射到另一个矢量空间的函数 $T: V \to W$。它需要满足下面的性质：

- $T(u + v) = T(u) + T(v)$。
- $T(av) = aT(v)$

数学中的线性变换常写成大写的拉丁字母；量子力学中，我们通常使用大写字母表示线性变换，如 $T$。此外，线性变换虽然是函数，在使用时通常不用圆括号将参数扩进来，因此后面我们会使用 $Tv$ 而非 $T(v)$ 的形式。为了用更加直观的方式表示线性变换，让我们考察其连接的两个矢量空间的基的关系。设矢量空间 $V$ 的基为 $v_1, \dots, v_m$，其通过 $T$ 映射到另一个矢量空间 $W$，后者的基为 $w_1, \dots, w_n$。根据线性变换的特征，我们可以将 $W$ 中的每个基写成下面的等式：

$$
\begin{equation*}
    w_k = T_{1k}v_1 + \dots + T_{mk}v_m
\end{equation*}
$$

其中每个 $T_{ij}, i=1,\dots,m, j=1,\dots,n$ 都是域上的线性变换（通常都是标量），其可以通过 $V$ 和 $W$ 以及它们基的选取唯一确定。综上，我们可以将线性变换 $T$ 写成下面的形式：

$$
\begin{equation*}
    \mathcal{M}_{\mathscr{B}_v\to \mathscr{B}_w}(T) = \begin{pmatrix} T_{11} & \dots & T_{1n} \\ \vdots & \ddots & \vdots \\ T_{m1} & \dots & T_{mn} \end{pmatrix}
\end{equation*}
$$

这个结构称为 **矩阵（Matrix）**。更具体来说，它是一个 $m\times n$（$m$ 行 $n$ 列）的矩阵。需要注意的是，相同的线性变换，在选取 $V$ 和 $W$ 中不同的基 $\mathscr{B}_v$ 与 $\mathscr{B}_w$ 时得到的矩阵形式一般不相同。不过在不考虑不同基的语境下，两者可以认为是等价的（后面我们会谈到理想的情形），此时我们不加区分地使用矩阵和线性变换，同时且不区分 $\mathcal{M}(T)$ 与 $T$。

细心的同学会发现，此前我们定义地左右矢在表示形式上和矩阵非常相似；事实正是如此：矢量可以看作矩阵的特殊情况（左矢是 $1\times n$ 的矩阵，而右矢是 $m\times 1$ 的矩阵），因此下面介绍的运算规则同样适用于矢量：

两个 $m\times n$ 矩阵的加法定义如下：

$$
\begin{equation*}
    T + S = \begin{pmatrix}
        T_{11} + S_{11} & \dots & T_{1n} + S_{1n} \\
        \vdots & \ddots & \vdots \\
        T_{m1} + S_{m1} & \dots & T_{mn} + S_{mn} 
    \end{pmatrix}
\end{equation*}
$$

矩阵的标量积定义如下：

$$
\begin{equation*}
    cT = \begin{pmatrix}
        cT_{11} & \dots & cT_{1n} \\
        \vdots & \ddots & \vdots \\
        cT_{m1} & \dots & cT_{mn}
    \end{pmatrix}
\end{equation*}
$$

定义零矩阵为所有元素均为 $0$ 的矩阵：

$$
\begin{equation*}
    0 = \begin{pmatrix}
         0 & \dots & 0 \\
         \vdots & \ddots & \vdots \\
         0 & \dots & 0 
    \end{pmatrix}
\end{equation*}
$$

在上面的设置下，我们不难发现 $m\times n$ 矩阵所在的空间也是矢量空间，记作 $M_{m\times n}$，而左矢和右矢所在的矢量空间只是其中的两个特例。

对于矢量空间 $M_{m\times \ell}$ 和 $M_{\ell\times n}$，定义 **矩阵乘法（Matrix Multiplication）** 如下：

$$
\begin{equation*}
    (TU)_{ij} = \sum_{k=1}^\ell T_{ik}U_{kj}, \qquad T \in M_{m\times \ell},\ U \in M_{\ell\times n},\ i=1,\dots,m,\ j=1,\dots,n
\end{equation*}
$$

可以看到这个结果是矢量空间 $M_{m\times n}$ 中的矩阵。可以证明，矩阵乘法满足结合律（即运算的先后顺序不影响结果），但一般不满足交换律。这个运算的主要意义在于，其将线性变换抽象成独立于矢量之外的存在。比如对矢量 $|\psi\rangle$ 先后进行变换 $T$ 和 $U$，从最开始的定义来看是 $U(T|\psi\rangle)$；通过抽象出矩阵和矩阵乘法，我们也可以将这个操作理解为两个线性变换的乘积 $UT$ 代表了新的变换，并作用于矢量 $|\psi\rangle$ 得到 $(UT)|\psi\rangle$。后面我们会认识到这种灵活性的好处。

`numpy.array` 同样可以用于存储矩阵（事实上，此前我们左右矢就存成了二维的数组，即矩阵），下面我们将演示矩阵的乘法操作：

In [4]:
T = np.array([[2+1j, 3+2j], [1, 0], [-1-1j, 2j]], dtype=complex)
U = np.array([[2+3j, 1, -3-2j], [0, 3+1j, 4-3j]], dtype=complex)
v = ket(1-2j, 0, 1+2j)
# 检测结合律。注意 numpy.array 的矩阵乘法使用的是 @，而 * 是完全不相关的含义！
print((T @ U) @ v)
print(T @ (U @ v))

[[ 47.+26.j]
 [  9. -9.j]
 [-28.+20.j]]
[[ 47.+26.j]
 [  9. -9.j]
 [-28.+20.j]]


### 内积空间

**内积空间（Inner Product Space）** 是定义了 **内积（Inner Product）** 运算 $\langle\cdot|\cdot\rangle: V\times V \to F$ 的矢量空间，其需要满足下面的性质：

- 交换后互为复数共轭，即 $\langle u|v \rangle = \langle v|u \rangle^*$。
- 与自身的内积为非负实数，即 $\langle v|v \rangle \ge 0$。
- 满足线性变换的要求，即 $\langle au|v + w \rangle = a\langle u|v \rangle + a\langle u|w \rangle$。

实际上从内积的记号就可以看出，互为对偶空间中的左矢和右矢之积就是我们想要的内积：

$$
\begin{equation*}
    \langle \alpha|\beta \rangle = \langle \alpha||\beta \rangle = a_1^*b_1 + \dots + a_n^*b_n
\end{equation*}
$$

特别地，我们将矢量 $|\psi\rangle$ 的 **范数（Norm）** 定义为其与自身之积的平方根：

$$
\begin{equation*}
    \lVert{\psi}\rVert = \sqrt{\langle \psi|\psi \rangle}
\end{equation*}
$$

根据内积的定义，这个值一定是非负实数。它的几何意义正是矢量的长度。范数为 $1$ 的矢量被称为 **归一化矢量（Normalized Vector）**。如果两个矢量的内积为零，则称它们相互 **正交（Orthogonal）**。特别地，如果一集矢量两两之间均正交，则称它们为 **标准正交集（Orthonormal Set）**。我们最常见到的标准正交集构成一个矢量空间的基，因此称其为 **标准正交基（Orthonormal Basis）**；之后出现的所有基，在没有特别说明的情况下都是标准正交的。举例来说，矢量空间 $\mathbb{C}^3$ 中的 $|e_x\rangle$、$|e_y\rangle$、$|e_z\rangle$ 就是一个标准正交基。为了在计算中方便表示，我们通常将标准正交基中任两个基向量的内积写成：

$$
\begin{equation*}
    \langle e_i|e_j \rangle = \delta_{ij} = \begin{cases}
        1 & i = j \\
        0 & i \ne j
    \end{cases}
\end{equation*}
$$

这里的 $\delta_{ij}$ 称为 **克罗内克记号（Kronecker Delta）**。通过这个记号的帮助，我们可以给出矢量在标准正交基中每个基矢量方向上的分量：

$$
\begin{equation*}
    a_i = \sum_{j=1}^n \delta_{ij}a_j = \sum_{j=1}^n \langle e_i|e_j \rangle a_j = \langle e_i| \sum_{j=1}^n a_j|e_j\rangle
    = \langle e_i|\alpha \rangle
\end{equation*}
$$

熟悉矢量分析的同学可以看到它和 $a_i = \hat{e}_i\cdot\mathbf{a}$ 的一致性；不过在内积空间中，我们不能随意调换内积两侧的矢量，即 $\langle e_i|\alpha \rangle \ne \langle \alpha|e_i \rangle$。 实际上：

$$
\begin{equation*}
    \langle \alpha|e_i \rangle = \langle e_i|\alpha \rangle^* = a_j^*
\end{equation*}
$$

值得一提的是，虽然很多时候我们将 $\mathbb{C}^n$ 中的标准正交基写成：

$$
\begin{equation*}
    \ket{e_1} = \begin{pmatrix} 1 \\ \vdots \\ 0 \end{pmatrix} \quad \dots \quad
    \ket{e_n} = \begin{pmatrix} 0 \\ \vdots \\ 1 \end{pmatrix}
\end{equation*}
$$

但这仅仅是以它们自身为基时，这些基矢量的矩阵形式。我们完全可以设置另一组正交基，如 $\mathbb{C}^2$ 中可以令：

$$
\begin{equation*}
    \ket{e_1'} = \frac{1}{\sqrt{2}}(\ket{e_1} + \ket{e_2}) = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix} \qquad
    \ket{e_2'} = \frac{1}{\sqrt{2}}(\ket{e_1} - \ket{e_2}) = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}} \end{pmatrix}
\end{equation*}
$$

它们是一组正交基（不难验证 $\braket{e_1'}{e_2'} = 0$），且满足我们此前提到的所有性质。对于任何用新的基表示的矢量，如：

$$
\begin{equation*}
    \ket{\alpha'} = \begin{pmatrix} 1 \\ -1 \end{pmatrix}_{\mathscr{B}'} = \ket{e_1'} - \ket{e_2'}
\end{equation*}
$$

我们可以将原来的基 $\mathscr{B}$ 表示成 $\mathscr{B}'$ 的线性组合：

$$
\begin{equation*}
    \ket{e_1} = \frac{1}{\sqrt{2}}(\ket{e_1'} + \ket{e_2'}) = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix}_{\mathscr{B}'} \quad 
    \ket{e_2} = \frac{1}{\sqrt{2}}(\ket{e_1'} - \ket{e_2'}) = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}} \end{pmatrix}_{\mathscr{B}'}
\end{equation*}
$$

即使在另一个基中，原来的基矢量依然满足此前的性质，如 $\braket{e_1}{e_2} = 0$，以及空间中任意矢量的投影（以 $\ket{\alpha} = 2\ket{e_1} - \ket{e_2}$ 为例）：

$$
\begin{align*}
    \braket{e_1}{\alpha} 
        &= \frac{1}{\sqrt{2}}(\bra{e_1'} + \bra{e_2'})\left[2\cdot\frac{1}{\sqrt{2}}(\ket{e_1'} + \ket{e_2'}) - \frac{1}{\sqrt{2}}(\ket{e_1'} - \ket{e_2'})\right] \\
        &= \frac{1}{2}(\bra{e_1'} + \bra{e_2'})(\ket{e_1'} + 3\ket{e_2'}) \\
        &= 2
\end{align*}
$$

这说明了矩阵表示不能唯一决定矢量本身的特征，且内积的结果与标准正交基的选取无关。下一节中我们会仔细说明矢量和矩阵在不同基中的变换。

有两个著名的不等式和内积相关，其一是 **施瓦兹不等式（Schwarz Inequality）**：

$$
\begin{equation*}
    |\langle \alpha|\beta \rangle|^2 \le \langle \alpha|\alpha \rangle\langle \beta|\beta \rangle
\end{equation*}
$$

如果从实数的矢量空间来看，这个不等式等价于 $|\cos\theta| \le 1$。不过在复空间中，我们最好求助于代数。注意到 $|\langle \alpha|\beta \rangle|^2 = \langle \alpha|\beta \rangle^* \langle \alpha|\beta \rangle$，设：

$$
\begin{equation*}
    |\gamma\rangle = |\beta\rangle - \frac{\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle}|\alpha\rangle
\end{equation*}
$$

（上面假设 $|\alpha\rangle \ne |0\rangle$，这种情况下施瓦兹不等式显然成立）考虑 $\langle \gamma|\gamma \rangle \ge 0$ 则有：

$$
\begin{align*}
    \langle \gamma|\gamma \rangle
        &= \left(\langle\beta| - \frac{\langle\alpha|\beta\rangle^*}{\langle\alpha|\alpha\rangle^*}\langle\alpha|\right)
           \left(|\beta\rangle - \frac{\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle}|\alpha\rangle\right) \\
        &= \langle\beta|\beta\rangle + \frac{\langle\alpha|\beta\rangle^*\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle^*\langle\alpha|\alpha\rangle}\langle\alpha|\alpha\rangle - \frac{\langle\alpha|\beta\rangle^*}{\langle\alpha|\alpha\rangle^*}\langle\alpha|\beta\rangle - \frac{\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle}\langle\beta|\alpha\rangle \\
        &= \langle\beta|\beta\rangle + \frac{\langle\alpha|\beta\rangle^*\langle\alpha|\beta\rangle}{\langle\alpha|\alpha\rangle^*} - \frac{\langle\alpha|\beta\rangle^*\langle\alpha|\beta\rangle}{\langle\alpha|\alpha^*\rangle} - \frac{\langle\alpha|\beta\rangle\langle\alpha|\beta\rangle^*}{\langle\alpha|\alpha\rangle} \\
        &= \langle\beta|\beta\rangle - \frac{|\langle\alpha|\beta\rangle|^2}{\langle\alpha|\alpha\rangle} \\
        &\ge 0
\end{align*}
$$

由于 $\langle\alpha|\alpha\rangle > 0$，将两边同时乘以 $\langle\alpha|\alpha\rangle$ 得到 $\langle\alpha|\alpha\rangle\langle\beta|\beta\rangle \ge |\langle\alpha|\beta\rangle|^2$，得证。

另外一个不等式是 **三角不等式（Triangle Inequality）**，即：

$$
\begin{equation*}
    \left\lVert\alpha + \beta\right\rVert \le \left\lVert\alpha\right\rVert + \left\lVert\beta\right\rVert
\end{equation*}
$$

考察不等式左侧的平方：

$$
\begin{align*}
    \left\lVert\ket{\alpha} + \ket{\beta}\right\rVert^2 
        &= \left\lVert\alpha\right\rVert^2 + \left\lVert\beta\right\rVert^2 + \langle\alpha|\beta\rangle + \langle\beta|\alpha\rangle \\
        &\le \left\lVert\alpha\right\rVert^2 + \left\lVert\beta\right\rVert^2 + 2\langle\alpha|\beta\rangle \\
        &\le \left\lVert\alpha\right\rVert^2 + \left\lVert\beta\right\rVert^2 + 2\left\lVert\alpha\right\rVert\left\lVert\beta\right\rVert \\
        &= (\left\lVert\alpha\right\rVert + \left\lVert\beta\right\rVert)^2
\end{align*}
$$

等于右侧平方，得证。

一如既往地，让我们用 `numpy.array` 探索矢量的内积运算：

In [5]:
# 由于 bra 和 ket 本质是矩阵，它们的积也是一个 1x1 的矩阵，我们应该返回其中的唯一元素
def inner(bra, ket):
    assert(is_bra(bra))
    assert(is_ket(ket))
    return (bra @ ket)[0][0]

a = bra(1j, 2)
b = ket(-1j, 2)
print(inner(a, b))

# 事实上，numpy.inner 可以对一维 numpy.array 进行类似内积的运算，不过它不会自动做复共轭，因此我们不采用这个写法
print(np.inner(np.array([-1j, 2j]), np.array([1, 2])))

(5+0j)
3j


### 间章：关于矢量和矩阵的形式

前面的小节中，我们已经了解了矢量、矩阵的定义；它们是线性代数的核心概念。简单来说，矢量代表了（矢量）空间中的一个点，而矩阵代表了从一个点（线性）映射到另一个点的规则。不过上节中介绍的基矢量的概念又让我们意识到：对于同一个点，可能有不同的矢量表示；同样，同一个线性变换在不同基的视角下会有不同的矩阵形式。从这个角度来说，一个点有无穷多种矢量形式，而一个线性变换也有无穷多种矩阵形式；点和线性变换是“唯一”的，但矢量和矩阵需要指定上下文（也就是基的选取）才能明确其定义。

本篇笔记中，我们总是选取标准正交基并将其无区别地写作：

$$
\begin{equation*}
    \ket{e_1} = \begin{pmatrix} 1 \\ \vdots \\ 0 \end{pmatrix} \qquad \dots \qquad
    \ket{e_n} = \begin{pmatrix} 0 \\ \vdots \\ 1 \end{pmatrix}
\end{equation*}
$$

这里的基矢量和 $\mathbb{C}^n$ 中的点没有任何关系，它们构成了任意 $n$ 维矢量空间中的一组特定的标准正交基（无所谓具体是哪一组）。当然，对于 $\mathbb{C}^n$ 的特别情形，我们乐于令 $\ket{e_i}$ 对应第 $i$ 根数轴上的单位矢量，从而使这个基张成的矢量表示和对应点的表示完全相同。

矩阵形式的选取同样由一个标准正交基决定。

在后面章节中我们会看到，量子力学中 *算符* 对应了线性代数中的线性变换，因此其矩阵形式同样取决于基。

### 矩阵的性质

本节让我们深入探讨矩阵。量子力学中，我们只关心从一个特定空间到其自身的线性变换。此时矩阵的行和列数量相同（这里的前提是研究的矢量空间为有限维度，量子力学中许多情况下这并不成立），我们将其称为 **方阵（Square Matrix）**。后文中不经特别说明，所有提到的矩阵都是方阵。对于一个已知的线性变换 $T$，为了得到它的矩阵形式，可以参考下面的等式（其中 $\ket{e_i}$ 是 $T_{ij}$ 所用的基矢量）：

$$
    T_{ij} = \sum_{k=1}^n\delta_{ik}T_{kj} = \sum_{k=1}^n \braket{e_i}{e_k} T_{kj} = \bra{e_i} \sum_{k=1}^n T_{kj}\ket{e_k} = \bracket{e_i}{T}{e_j}
$$

整个矩阵则可以通过下面的等式给出：

$$
\begin{equation*}
    T = \sum_{i=1}^n\sum_{j=1}^n T_{ij}\ket{e_i}\bra{e_j}
\end{equation*}
$$

下面我们将定义一些矩阵的操作以及特殊矩阵。

#### 转置矩阵

矩阵 $T$ 的 **转置（Transpose）** 定义为其关于主对角线 $T_{ii}$ 的镜像：

$$
\begin{equation*}
    (T^T)_{ij} = T_{ji}
\end{equation*}
$$
    
若矩阵的转置和它自身相同，称该矩阵 **对称（Symmetric）**；若其转置满足 $T^T = -T$，则称其为 **反对称（Anti-Symmetric）** 的。

#### 共轭矩阵

矩阵的 **复共轭（Complex Conjugate）** 定义为将其每个元素取复共轭的结果：

$$
\begin{equation*}
    (T^*)_{ij} = (T_{ij})^*
\end{equation*}
$$

若矩阵的复共轭和它自身相同，称该矩阵为 **实（Real）** 矩阵；若其复共轭满足 $T^* = -T$，则称其为 **虚（Imaginary）** 矩阵。

#### 共轭转置矩阵

矩阵的 **共轭转置（Conjugate Transpose）** 是其共轭的转置，记作 $T^\dagger$。值得一提的是，同一矢量的左矢和右矢互为共轭转置。若矩阵的共轭转置等于其自身，则称其为 **厄米特（Hermitian）** 矩阵。若其共轭转置满足 $T^\dagger = -T$，则称其为 **斜厄米特（Skew Hermitian）** 矩阵。如果矩阵满足 $T^\dagger T = TT^\dagger$，我们称其为 **正规矩阵（Normal Matrix）**。显然，厄米特矩阵都是正规矩阵。

#### 协因数阵

矩阵的 **协因数阵（Cofactor Matrix）** $C$ 为其 **协因数（Cofactor）**（代数余子式） $C_{ij}$ 组成的矩阵。对于任一个元素 $T_{ij}$，其协因数定义为去除 $i$ 行 $j$ 列得到的子矩阵的 **行列式（Determinant）** 乘以 $(-1)^{i+j}$。这里的行列式定义为：

$$
\begin{equation*}
    \text{det}(T) = \sum_{j=1}^nC_{ij}a_{ij}
\end{equation*}
$$   

其中 $i$ 是任取的。协因数和行列式的定义是相互引用的，其基本情形是 $1\times1$ 矩阵，此时它的行列式是其唯一的元素。
    
#### 单位矩阵
    
**单位矩阵（Identity Matrix）** 定义为除了主对角线上均为 $1$ 外，其它元素均为 $0$ 的矩阵，其满足 $IT = TI = T$。可以如下表示：
$$
\begin{equation*}
    I_{ij} = \delta_{ij}
\end{equation*}
$$

我们也可以通过任一组基表示出单位矩阵：

$$
\begin{equation*}
    I = \sum_{i=1}^n \ket{e_i}\bra{e_i}
\end{equation*}
$$

验证上式的正确性不难；考察 $IT$ 的第 $k$ 列，设其为 $|\tau\rangle$：

$$
\begin{align*}
    I\ket{\tau} 
        &= \left(\sum_{i=1}^n\ket{e_i}\bra{e_i}\right)\sum_{i=1}^nt_i|e_i\rangle \\
        &= \sum_{i=1}^n\sum_{j=1}^nt_j\ket{e_i}\braket{e_i}{e_j} \\
        &= \sum_{i=1}^n\delta_{ij}\sum_{j=1}^nt_j\ket{e_i} \\
        &= \sum_{i=1}^nt_i\ket{e_i} = \ket{\tau}
\end{align*}
$$

因此 $I$ 对 $T$ 的每一列都是单位变换，最终整个矩阵也必然是单位变换。值得一提的是，形如 $\ket{\psi}\bra{\psi}$ 的矩阵称为 **投影矩阵（Projection Matrix）**，其用于计算任一个矢量在 $\ket{\psi}$ 方向上的分量。
    
#### 逆矩阵
    
矩阵的 **逆（Inverse）** 是与其自身相乘得到单位矩阵的唯一矩阵。它可以通过协因数阵得到：  

$$
\begin{equation*}
    T^{-1} = \frac{1}{\text{det}(T)}C
\end{equation*}
$$

由于行列式出现在分母，可以预见当 $\text{det}(T) = 0$ 时，存在逆的矩阵称为 **可逆矩阵（Invertible Matrix）**，反之称为 **奇异矩阵（Singular Matrix）**。

#### 幺正矩阵
    
若矩阵的逆与其共轭转置相同，则称其为 **幺正矩阵（Unitary Matrix）**。此时有 $T^\dagger T = I$。注意到幺正矩阵也是正规矩阵。

两个矩阵之积的转置、共轭转置以及逆满足类似的运算规律：

$$
\begin{equation*}
    (TS)^T = S^TT^T \qquad
    (TS)^\dagger = S^\dagger T^\dagger \qquad
    (TS)^{-1} = S^{-1}T^{-1}
\end{equation*}
$$

前两个可以通过其定义证明；第三个式子可以由 $(S^{-1}T^{-1})(TS) = S^{-1}(T^{-1}T)S = S^{-1}S = I$ 以及逆矩阵的唯一性得到。

现在让我们探索如何从一个基线性变换到另一个基。设原来的基是 $|e_i\rangle$，新的基是 $|e_i'\rangle$。由于在同一个矢量空间中，我们总能找到一系列标量 $T_{ij}$ 使得：

$$
\begin{equation*}
    |e_i\rangle = T_{i1}|e_1'\rangle + \dots + T_{in}|e_n'\rangle
\end{equation*}
$$

此时对任意的 $\ket{\alpha} = \sum_{i=1}^na_i|e_i\rangle$，我们都有：

$$
\begin{equation*}
    a_i\ket{e_i} = a_i\sum_{j=1}^nT_{ij}\ket{e_j'}
\end{equation*}
$$

因此有：

$$
\begin{equation*}
    \ket{\alpha'} = T\ket{\alpha}
\end{equation*}
$$

对于作用在原来的基上的线性变换 $\mathcal{O}$，我们不难得到其在新的基上的形式：

$$
    \mathcal{O'} = T\mathcal{O}T^{-1}
$$

这是因为变换一个 $|e_i'\rangle$ 上的矢量等价于将其转换成 $\ket{e_i}$ 上的矢量，进行 $T$ 变换，然后重新转换为 $\ket{e_i'}$ 上的矢量。这样的两个矩阵 $\mathcal{O}$ 和 $\mathcal{O'}$ 被称为 **相似（Similar）** 的（因为它们本质做的是完全相同的线性变换，只不过作用于不同的基）。

In [40]:
def I(n):
    # np.identity 生成单位矩阵
    return np.identity(n, dtype=complex)

T = np.array([[1+2j, 2, 3], [-1, -2-1j, 3], [1, -2+3j, 3]])

print(f'T\n {T}')
print(f'Conjugate of T\n {T.conj()}')
print(f'Conjugate transpose of T\n {dagger(T)}')

# numpy.linalg.inv 返回矩阵的逆
U = np.linalg.inv(T)
print(f'Inverse of T\n {U}')

# 由于数值计算存在误差，这个结果不是精确的 I
print(f'Identity matrix?\n {T @ U}')

def almost_same(x, y, tolerance=1e-10):
    if type(x) == float or type(y) == float:
        return abs(x - y) < tolerance
    if type(x) == np.ndarray and type(y) == np.ndarray:
        diff = x - y
        return (abs(diff) < tolerance).all()
    return False

# 可以看到，T @ U 确实基本上等于 I
print(f'But that is actually near: TT^{{-1}} ~ I?{chr(10)} {almost_same(T @ U, I(3))}')

T
 [[ 1.+2.j  2.+0.j  3.+0.j]
 [-1.+0.j -2.-1.j  3.+0.j]
 [ 1.+0.j -2.+3.j  3.+0.j]]
Conjugate of T
 [[ 1.-2.j  2.-0.j  3.-0.j]
 [-1.-0.j -2.+1.j  3.-0.j]
 [ 1.-0.j -2.-3.j  3.-0.j]]
Conjugate transpose of T
 [[ 1.-2.j -1.-0.j  1.-0.j]
 [ 2.-0.j -2.+1.j -2.-3.j]
 [ 3.-0.j  3.-0.j  3.-0.j]]
Inverse of T
 [[ 0.08219178-0.21917808j -0.28082192+0.08219178j  0.19863014+0.1369863j ]
 [ 0.10958904+0.04109589j -0.04109589+0.10958904j -0.06849315-0.15068493j]
 [ 0.08675799-0.00913242j  0.17579909+0.08675799j  0.07077626-0.07762557j]]
Identity matrix?
 [[ 1.00000000e+00+6.93889390e-18j  0.00000000e+00-5.55111512e-17j
   5.55111512e-17-5.55111512e-17j]
 [-5.55111512e-17+6.24500451e-17j  1.00000000e+00+0.00000000e+00j
   0.00000000e+00+2.77555756e-17j]
 [ 5.55111512e-17-4.85722573e-17j  0.00000000e+00+0.00000000e+00j
   1.00000000e+00-2.77555756e-17j]]
But that is actually near: TT^{-1} ~ I?
 True


### 特征值与特征矢量

对于任意的矩阵 $T$，我们总能找到一些特殊的矢量 $\ket{\alpha}$ 使得下式成立：

$$
\begin{equation*}
    T\ket{\alpha} = \lambda\ket{\alpha}
\end{equation*}
$$

这样的矢量被称为 $T$ 的 **特征矢量（Eigenvector）**，而对应的 $\lambda$ 被称为 **特征值（Eigenvalue）**。为了求出矩阵的特征矢量，可以列出下式：

$$
\begin{equation*}
    (T - \lambda I)\ket{\alpha} = \ket{0} \implies \text{det}(T - \lambda I) = 0
\end{equation*}
$$

之后经过整理我们可以得到一个 $n$ 次的方程：

$$
\begin{equation*}
    c_n\lambda^n + \dots + c_0 = 0
\end{equation*}
$$

根据代数基本定理，上面的方程一定能找到 $n$ 的复数解 $\lambda_1, \dots, \lambda_n$，称为矩阵 $T$ 的 **波谱（Spectrum）**。一些情况下，波谱中会出现重复的特征值，此时我们称其为 **退化（Degenerate）** 的波谱。当一个特征值重复 $d$ 次时，我们称其拥有 **重数（Multiplicity）** $d$。不难认识到 $\sum_{i=1}^nd_i = \text{dim}(V)$。

当一个波谱可以张成整个矢量空间时，我们可以尝试将其对应特征矢量作为矢量空间的基。观察到：

$$
\begin{equation*}
    T\sum_{i=1}^n a_i\ket{e_i} = \sum_{i=1}^n \lambda_i\ket{e_i} = \begin{pmatrix}
        \lambda_1 & \dots & 0 \\
        \vdots & \ddots & 0 \\
        0 & \dots & \lambda_n
    \end{pmatrix}
\end{equation*}
$$

我们将其称为线性变换 $T$ 的 **对角化形式（Diagonal Form）**，这样仅在主对角线上有非零元素的矩阵称为 **对角矩阵（Diagonal Matrix）**。特征矢量能张成矢量空间的矩阵被称为 **可对角化（Diagonalizable）** 的，因为它拥有对角化形式。对角化的矩阵因为其简单性，我们在计算中会青睐于使用这种形式。

定义矩阵的 **迹（Trace）** 为其主对角线上元素之和，即：

$$
\begin{equation*}
    \text{tr}(T) = \sum_{i=0}^nT_{ii}
\end{equation*}
$$

矩阵迹有下面的一些重要特征：

- 它是线性的，因此 $\text{tr}(aT + U) = a\text{tr}(T) + \text{tr}(U)$。
- 它有循环性，比如 $\text{tr}(TUS) = \text{tr}(UST) = \text{tr}(STU)$。
- 它与基的选取无关。对于任选的两个基 $\ket{u_i}$ 和 $\ket{v_i}$ 都有：
    
$$
\begin{equation*}
    \text{tr}(T) = \sum_{i=1}^n \bracket{u_i}{T}{u_i} = \sum_{i=1}^n \bracket{v_i}{T}{v_i}
\end{equation*}
$$

- 它等于矩阵所有特征值受重数的加权和：

$$
\begin{equation*}
    \text{tr}(T) = \sum_{i=1}^n d_i\lambda_i
\end{equation*}
$$
    
这里值得再提一下行列式。我们此前为了定义逆矩阵，用协因数阵辅助定义了行列式。事实上通过特征值，我们可以得到更加优雅的定义。矩阵的行列式为其特征值受重数的加权积，即：

$$
\begin{equation*}
    \text{det}(T) = \prod_{i=1}^n \lambda_i^{d_i}
\end{equation*}
$$
    
行列式在量子力学中的用处不多，因此在这里就不深入探索了。

### 正规矩阵

本节介绍在量子力学与量子计算中有特别地位的厄米特矩阵和幺正矩阵。首先让我们了解正规矩阵的性质。回忆正规矩阵是满足 $TT^\dagger = T^\dagger T$ 的矩阵。

- 正规矩阵和它的共轭转置作用于任意矢量，其结果的范数相同：

$$
\begin{equation*}
    \braket{\alpha}{(T^\dagger T - TT^\dagger)\alpha} = 0 \implies
    \braket{\alpha}{T^\dagger T\alpha} = \braket{\alpha}{TT^\dagger\alpha} \implies
    \left\lVert T\alpha\right\rVert^2 = \left\lVert T^\dagger\alpha\right\rVert^2
\end{equation*}
$$
- 正规矩阵和它的共轭转置拥有相同的特征矢量。首先不难验证 $T - \lambda I$ 也是正规矩阵，因此：

$$
\begin{equation*}
    0 = \left\lVert(T - \lambda I)\alpha\right\rVert = \left\lVert(T - \lambda I)^\dagger\alpha\right\rVert = \left\lVert(T^\dagger - \lambda^*I)\alpha\right\rVert
\end{equation*}
$$ 
- **波谱定理（Spectral Theorem）**：对于矢量空间 $V$ 上的线性变换 $T$ 是正规矩阵，当且仅当 $T$ 的特征矢量是 $V$ 的一个标准正交基。

#### 厄米特矩阵

回忆厄米特矩阵是共轭转置等于其自身的矩阵。它广泛出现于量子力学中，并满足一个重要的等式关系：

$$
\begin{equation*}
    \braket{T\alpha}{\beta} = \braket{\alpha}{T\beta}
\end{equation*}
$$

注意上面的记号 $\bra{T\alpha}$ 指的是 $T\ket{\alpha}$ 对应的左矢。下面是一个简单的证明：

$$
\begin{equation*}
    \braket{T\alpha}{\beta} = (T^\dagger\ket{\alpha})^\dagger\ket{\beta} = \ket{\alpha}^\dagger (T\ket{\beta}) = \braket{\alpha}{T\beta}
\end{equation*}
$$

厄米特矩阵还有一些重要的性质：

- 其特征值一定是实数。设其特征矢量为 $\ket{\alpha}$，则：
$$
\begin{equation*}
    \braket{\alpha}{T\alpha} = \braket{\alpha}{\lambda\alpha} = \lambda\sbraket{\alpha}
\end{equation*}
$$
    同时根据此前推得的等式：  
$$
\begin{equation*}
    \braket{\alpha}{T\alpha} = \braket{T\alpha}{\alpha} = \braket{\lambda\alpha}{\alpha} = \lambda^*\sbraket{\alpha}
\end{equation*}
$$
    因此有 $\lambda = \lambda^*$，故特征值为实数。反过来说，特征值为实数的矩阵一定是厄米特矩阵。
    
- 其不同特征值对应的特征矢量相互正交。设特征矢量 $\ket{\alpha}$ 和 $\ket{\beta}$ 分别对应不同的特征值 $\lambda$ 与 $\mu$，则：
$$
\begin{equation*}
    \braket{\alpha}{T\beta} = \braket{\alpha}{\mu\beta} = \mu\braket{\alpha}{\beta}
\end{equation*}
$$
    同时：  
$$
\begin{equation*}
    \braket{\alpha}{T\beta} = \braket{T\alpha}{\beta} = \braket{\lambda\alpha}{\beta} = \lambda^*\braket{\alpha}{\beta}
\end{equation*}
$$
    由于 $\lambda$ 一定是实数，这里如果 $\braket{\alpha}{\beta} \ne 0$ 我们就得到 $\mu = \lambda^* = \lambda$，与题设矛盾。因此一定有 $\ket{\alpha}$ 和 $\ket{\beta}$ 正交。
    
- 其是可对角化的，因为它是正规矩阵。

#### 幺正矩阵

回忆幺正矩阵为逆等于共轭转置的矩阵，即 $TT^\dagger = I$。它的部分特点如下：

- 其行列式绝对值为 $1$。考虑行列式的运算特性：
$$
\begin{equation*}
    |\text{det}(T)|^2 = \text{det}(T)[\text{det}(T)]^* = \text{det}(TT^\dagger) = \text{det}(I) = 1 
\end{equation*}
$$
    因此幺正矩阵的行列式，其绝对值为 $1$。特别地，如果它同时是哈密顿矩阵，其行列式只能是 $\pm 1$。

- 幺正矩阵同时作用于内积的左右矢时不影响结果，即：
$$
\begin{equation*}
    \braket{U\alpha}{U\beta} = (U\ket{\alpha})^\dagger(U\ket{\beta}) = \bra{\alpha}U^\dagger U\ket{\beta} = \braket{\alpha}{\beta}
\end{equation*}
$$
- 其特征值的绝对值为 $1$。证明如下：
$$
\begin{equation*}
    |\lambda|^2\sbraket{\alpha} = \sbraket{\lambda\alpha} = \sbraket{U\alpha} = \sbraket{\alpha}
\end{equation*}
$$
- 其是可对角化的，因为它是一个正规矩阵。

若 $U$ 同为厄米特矩阵且幺正矩阵，我们得到一个有趣的性质：

$$
\begin{equation*}
    U^2 = U^\dagger U = U^{-1}U = I
\end{equation*}
$$

此时我们可以尝试将矩阵运算拓展到一些常用函数上，得到相当特别的结果：

- 指数函数 $e^x$。考虑 $e^{i\theta U}$，使用 $x=0$ 处的泰勒展开：

$$
\begin{align*}
    e^{i\theta U}
        &= I + i\theta U + \frac{1}{2!}(i\theta)^2U^2 + \frac{1}{3!}(i\theta)^3U^3 + \dots \\
        &= I\left( 1 - \frac{1}{2!}\theta^2 + \frac{1}{4!}\theta^4 - \dots \right) + iU\left( \theta - \frac{1}{3!}\theta^3 + \frac{1}{5!}\theta^5 - \dots \right) \\
        &= I\cos\theta + iU\sin\theta
\end{align*}
$$

### 其它特殊矩阵类型

#### 正矩阵

**正矩阵（Positive Matrix）**（更加通用的称呼是 **正算符（Positive Operator）**，我们后面会提到量子力学中算符和矩阵的关系）定义为对任意矢量 $\ket{\psi}$ 都满足下面关系的矩阵：

$$
\begin{equation*}
    \bracket{\psi}{T}{\psi} \ge 0
\end{equation*}
$$

（显然一个更好的名字是非负矩阵，因为上面的结果并不总是正数）对于正矩阵，我们总能找到其 **平方根（Square Root）** $R$ 满足 $R^2 = T$。事实上，每个正矩阵都有一个平方根同时也是正矩阵（这和非负实数域的平方根性质完全相同）。

#### 等距矩阵

**等距矩阵（Isometric Matrix）** 指的是保持矢量范数不变的矩阵：

$$
\begin{equation*}
    \lVert{T\psi}\rVert = \lVert{\psi}\rVert
\end{equation*}
$$

### 张量积

矩阵的 **张量积（Tensor Product）**，或 **克罗内克积（Kronecker Product）** 定义如下：

$$
\begin{equation*}
    A\otimes B = \begin{pmatrix}
        A_{11}B & \dots & A_{nn}B \\
        \vdots & \ddots & \vdots \\
        A_{n1}B & \dots & A_{nn}B
    \end{pmatrix} = \begin{pmatrix}
        A_{11}B_{11} & \dots & A_{11}B_{1n} & \dots & A_{1n}B_{11} & \dots & A_{1n}B_{nn} \\
        \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots \\
        A_{11}B_{n1} & \dots & A_{11}B_{nn} & \dots & A_{1n}B_{n1} & \dots & A_{1n}B_{nn} \\
        \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots \\
        A_{n1}B_{11} & \dots & A_{n1}B_{1n} & \dots & A_{nn}B_{11} & \dots & A_{nn}B_{nn} \\
        \vdots & \ddots & \vdots & \ddots & \vdots & \ddots & \vdots \\
        A_{n1}B_{n1} & \dots & A_{n1}B_{nn} & \dots & A_{nn}B_{n1} & \dots & A_{nn}B_{nn} \\
    \end{pmatrix}
\end{equation*}
$$

对于维度为 $N_1$ 和 $N_2$ 的两个矩阵，其张量积的维度为 $N_1N_2$。根据定义我们不难发现张量积是一个满足结合律的运算符，且满足分配律和标量积的结合律，这里不展开说明了。

特别地，若 $A$ 是一个右矢 $\ket{\alpha}$，$B$ 是一个左矢 $\bra{\beta}$，张量积等同于：

$$
\begin{equation*}
    \ket{\alpha}\otimes \bra{\beta} = \ket{\alpha}\bra{\beta}
\end{equation*}
$$

当两个矩阵都是右矢时，我们会在左右矢符号中将张量积符号省略，写作：

$$
\begin{equation*}
    \ket{\psi\phi} = \ket{\psi}\otimes \ket{\phi}
\end{equation*}
$$

注意这里的简写并不如最开始提到的那种直观；$\ket{\psi + \phi} \equiv \ket{\psi} + \ket{\phi}$、$\ket{c\psi} = c\ket{\psi}$，然而 $\ket{\psi\phi} = \ket{\psi}\otimes \ket{\phi}$，需要适应一下（一些人会直接将张量积完全省略，变为 $\ket{\psi}\ket{\phi}$，但这和矩阵积的表示方法发生冲突，在一些情况下会造成歧义，因此我不采用这个写法）。

虽然张量积看起来将两个矩阵的内容完全混杂起来了，但在实际使用时两者可以很清晰地区分。考虑 $M_1 = A_1\otimes B_1$ 而 $M_2 = A_2\otimes B_2$，此时它们的矩阵积为：

$$
\begin{equation*}
    M_1M_2 = (A_1\otimes B_1)(A_2\otimes B_2) = (A_1A_2)\otimes (B_1B_2)
\end{equation*}
$$

上面这个公式非常重要，我们会在后面量子电路中频繁使用。特别地，对于左矢和右矢的运算，我们有：

$$
\begin{equation*}
    \braket{\psi_1\phi_1}{\psi_2\phi_2} = \braket{\psi_1}{\psi_2}\braket{\phi_1}{\phi_2}
\end{equation*}
$$

且对于矩阵 $T_1$ 和 $T_2$，我们可以将其作用于 $\ket{\psi} = \ket{\psi_1\psi_2}$ 等价写为：

$$
\begin{equation*}
    (T_1\otimes T_2)\ket{\psi} = (T_1\otimes T_2)(\ket{\psi_1\psi_2}) = (T_1\ket{\psi_1})\otimes (T_2\ket{\psi_2}) = \ket{T_1\psi_1\,T_2\psi_2}
\end{equation*}
$$

张量积会保留矩阵的一些好性质。比如：

- 若 $A$、$B$ 均为厄米特矩阵，则 $A\otimes B$ 也是厄米特矩阵。
- 若 $A$、$B$ 均为幺正矩阵，则 $A\otimes B$ 也是幺正矩阵。

下面是证明。对于 $A$ 可以作用的状态 $\ket{\alpha_1}$ 和 $\ket{\alpha_2}$ 以及 $B$ 可以作用的状态 $\ket{\beta_1}$ 和 $\ket{\beta_2}$，记 $C = A\otimes B$、$\ket{\psi_1} = \ket{\alpha_1\beta_1}$ 且 $\ket{\psi_2} = \ket{\alpha_2\beta_2}$，则：

$$
\begin{equation*}
    C\ket{\psi_2} = (A\ket{\alpha_2})\otimes(B\ket{\beta_2})
\end{equation*}
$$

因而：

$$
\begin{equation*}
    \bracket{\psi_1}{C}{\psi_2} = (\bra{\alpha_1}\otimes\bra{\beta_1})[(A\ket{\alpha_2})\otimes(B\ket{\beta_2})] = \bracket{\alpha_1}{A}{\beta_1}\bracket{\alpha_2}{B}{\beta_2}
\end{equation*}
$$

现在考察 $\bracket{\psi_1}{C}{\psi_2}^\dagger$：

$$
\begin{align*}
    \bracket{\psi_1}{C}{\psi_2}^\dagger 
        &= [\bracket{\alpha_1}{A}{\beta_1}\bracket{\alpha_2}{B}{\beta_2}]^\dagger \\
        &= \bracket{\alpha_2}{B}{\beta_2}^\dagger \bracket{\alpha_1}{A}{\beta_1}^\dagger \\
        &= \bracket{\beta_2}{B}{\alpha_2} \bracket{\beta_1}{A}{\alpha_1} \\
        &= (\bra{\beta_2}\otimes\bra{\beta_1})(B\ket{\alpha_2}\otimes A\ket{\alpha_1}) \\
        &= \bra{\psi_2}(B\otimes A)\ket{\psi_1} \\
        &= \bracket{\psi_2}{C}{\psi_1}
\end{align*}
$$

因此 $A\otimes B$ 确实是厄米特矩阵。

## 量子力学简介

### 波函数与算符

经典力学中，我们可以根据时间确定物体的准确位置，即通过函数 $x(t)$ 描述物体（本篇笔记中我们只考虑一维的空间）。但是实验证明，在微观情况下，**量子（Quantum）**，即物体的最小单元，其在某一刻的位置是由分布 $\psi(x)$ 决定的。对于任意一点 $x_0$ 上其出现的概率密度为 $|\psi(x_0)|^2$。由于概率加和应为 $1$，我们有：

$$
\begin{equation*}
    \int_{-\infty}^\infty |\psi(x_0)|^2\,dx = 1
\end{equation*}
$$

特别地，粒子在 $a \le x \le b$ 上出现的概率为：

$$
\begin{equation*}
    \text{Pr}(a \le x \le b) = \int_a^b |\psi(x_0)|^2\,dx
\end{equation*}
$$

从统计角度出发，粒子所在位置的期望值为：

$$
\begin{equation*}
    \exp{x} = \int_{-\infty}^\infty x|\psi(x)|^2\,dx
\end{equation*}
$$

但如果我们需要的是例子的动量，以目前已有的公式似乎推断不出（$p$ 是一个和 $x$ 相关的量）。为了能够继续，让我们首先对这个神秘的分布进行建模。首先，我们假定波函数可以被通过某个平面波描述（根据波粒二象性）：

$$
    \Psi(x, t) = Ae^{i(kx - \omega t)}
$$

其中 $k$ 时 **波数（Wave Number）**、$\omega$ 是 **角频率（Angular Frequency）**。此时它的偏导数为：

$$
\begin{equation*}
    \frac{\partial \Psi}{\partial t} = -i\omega\Psi(x, t) \qquad \frac{\partial^2 \Psi}{\partial x^2} = -k^2\Psi(x, t)
\end{equation*}
$$

根据波数的定义 $k = \frac{2\pi}{\lambda}$ 以及德布罗意公式的变形 $p = \frac{2\pi\hbar}{\lambda}$，我们有：

$$
\begin{equation*}
    k = \frac{p}{\hbar}
\end{equation*}
$$

哈密顿量 $H = \frac{p^2}{2m} + V(x, t)$，考察 $H\Psi(x, t)$ 有：

$$
\begin{align*}
    H\Psi(x, t)
        &= \frac{p^2}{2m}\Psi(x, t) + V(x, t)\Psi(x, t) \\
        &= -\frac{\hbar^2}{2m}\frac{\partial^2 \Psi}{\partial x^2} + V(x, t)\Psi(x, t)
\end{align*}
$$

对于粒子同时有 $H = \hbar\omega$，因此上面等式左侧变形为 $i\hbar\frac{\partial \Psi}{\partial t}$。最后我们得到：

$$
\begin{equation*}
    i\hbar\frac{\partial \Psi}{\partial t} = -\frac{\hbar^2}{2m}\frac{\partial^2\Psi}{\partial x^2} + V(x, t)\Psi
\end{equation*}
$$

这便是鼎鼎大名的 **薛定谔方程（Schrodinger's Equation）**。其用于描述量子力学中所有的波函数。注意虽然上面我们看似“推导”出来了薛定谔方程，但其假设是比较刻意的。我们完全可以这个方程看作公理（当然实际上更适合作为公理的是我们想要得到的动量的表达式，或者是在后续小节中介绍的标准对易关系；通过这两个中任一个都可以相对轻松地推出薛定谔方程）。

现在我们可以尝试计算动量的期望了。回忆 $p = m\frac{dx}{dt}$，因此：

$$
\begin{equation*}
    \frac{d\exp{x}}{dt} = \frac{d}{dt} \int x|\Psi|^2\,dx = \int x \frac{\partial}{\partial t}\left(\Psi^*\Psi\right)\,dx = \int x\left(\frac{\partial\Psi^*}{\partial t} + \frac{\partial \Psi}{\partial t}\right)\,dx
\end{equation*}
$$

根据薛定谔方程，我们有：

$$
\begin{equation*}
    \frac{\partial\Psi}{\partial t} = \frac{i\hbar}{2m}\frac{\partial^2\Psi}{\partial x^2} - \frac{i}{\hbar}V\Psi \qquad
    \frac{\partial\Psi^*}{\partial t} = -\frac{i\hbar}{2m}\frac{\partial^2\Psi^*}{\partial x^2} + \frac{i}{\hbar}V\Psi^*
\end{equation*}
$$

代入原来的式子：

$$
\begin{equation*}
    \frac{d\exp{x}}{dt} = -\frac{i\hbar}{2m}\int\left(\Psi^*\frac{\partial\Psi}{\partial x} - \frac{\partial\Psi^*}{\partial x}\Psi\right)\,dx = -\frac{i\hbar}{m}\int\Psi^*\frac{\partial\Psi}{\partial x}\,dx
\end{equation*}
$$

因此：

$$
\begin{equation*}
    \exp{p} = -i\hbar\int\Psi^*\frac{\partial\Psi}{\partial x}\,dx
\end{equation*}
$$

和位置的期望值对比，我们会发现它们都有一个特征，即：

$$
\begin{equation*}
    \exp{Q} = \int\Psi^*[\hat{Q}]\Psi\,dx
\end{equation*}
$$

对于位置来说，我们有 $\hat{x} = x$，而对于动量，则有：

$$
\begin{equation*}
    \hat{p} = -i\hbar\frac{\partial}{\partial x}
\end{equation*}
$$

一些教材将这个作为基本的公理并用于推导薛定谔方程；不过由于它比较魔幻，我还是将其作为结论而非假设了。

我们将这些在量子力学中表示物理量的期望特征的对象成为 **算符（Operator）**，用物理量上加尖角符号的形式表示。读者可能认为这和矩阵的表示方式重复了，但下一节我们就会提到两者之间的联系。所有算符都能表示成 $\hat{x}$ 和 $\hat{p}$ 两个算符的表达式：

$$
\begin{equation*}
    \hat{Q} = \hat{Q}(\hat{x}, \hat{p}) = \hat{Q}\left(\hat{x}, -i\hbar\frac{\partial}{\partial x}\right)
\end{equation*}
$$

举例来说，经典力学中的动能满足：

$$
\begin{equation*}
    T = \frac{p^2}{2m}
\end{equation*}
$$

因此其算符便是：

$$
\begin{equation*}
    \hat{T} = -\frac{1}{2m}\frac{\partial^2}{\partial x^2}
\end{equation*}
$$

#### 静止状态

本篇中，我们几乎总是假设薛定谔方程中的 $V$ 与时间无关，此时我们可以通过 **分离变量法（Separation of Variables）** 得到解，只需设波函数可以表示成两个一元函数的乘积：

$$
\begin{equation*}
    \Psi(x, t) = \psi(x)\phi(t)
\end{equation*}
$$

如果将 $\psi(x)$ 和 $\phi(t)$ 代入薛定谔方程，就能得到：

$$
\begin{equation*}
    i\hbar\frac{1}{\phi}\frac{d\phi}{dt} = -\frac{\hbar^2}{2m}\frac{1}{\psi}\frac{d^2\psi}{dx^2} + V
\end{equation*}
$$

为了将上面的两个常微分方程分开，令 $E = i\hbar\dfrac{1}{\phi}\dfrac{d\phi}{dt}$，这样就有：

$$
\begin{equation*}
    -\frac{\hbar^2}{2m}\frac{d^2\psi}{dx^2} + V\psi = E\psi
\end{equation*}
$$

其被称为 **时间无关的薛定谔方程（Time-independent Schrodinger Equation）**，其通解和 $V(x)$ 有关；另外一个是时间相关项：

$$
\begin{equation*}
    \frac{d\phi}{dt} = -\frac{iE}{\hbar}\phi
\end{equation*}
$$

它的解非常基础：

$$
\begin{equation*}
    \phi(t) = e^{-iEt/\hbar}
\end{equation*}
$$

因此，一个比较典型的量子力学题目设置是，给定一个势能分布 $V(x)$，求系统的波函数。此时需要先解出事件无关薛定谔方程的解 $\psi(x)$，随机乘上一个时间的摆动系数，就得到：

$$
\begin{equation*}
    \Psi(x, t) = \psi(x)e^{iEt/\hbar}
\end{equation*}
$$

这个摆动系数不会干扰 **静止状态（Stationary State）** $\psi(x)$ 的归一性，即：

$$
\begin{equation*}
    |\Psi(x, t)|^2 = \Psi^*\Psi = \psi^* e^{iEt/\hbar}\psi e^{-iEt/\hbar} = \psi^*\psi = |\psi(x)|^2
\end{equation*}
$$

同时，注意到所有其它算符 $\hat{Q}(\hat{x}, \hat{p})$ 都与时间无关，因此物理量的期望：

$$
\begin{equation*}
    \exp{Q(x, p)} = \int\psi^*Q\left(x, -i\hbar\frac{d}{dx}\right)\psi\,dx
\end{equation*}
$$

也和时间无关。

### 希尔伯特空间

本节中，我们会揭示量子力学和线性代数的联系。对于任意波函数 $\Psi$，其必须是 **归一化（Normalized）** 的，即：

$$
\begin{equation*}
    \int_{-\infty}^\infty |\Psi|^2\,dx = 1
\end{equation*}
$$

如果我们将情况一般化，其相当于要求一个函数在特定区间中 **平方可积（Square-Integrable）**：

$$
\begin{equation*}
    \int_a^b |\Psi|^2\,dx < \infty
\end{equation*}
$$

我们将所有满足这样要求的函数组成的空间称为 **希尔伯特空间（Hilbert Space）**，数学上记作 $L^2(a, b)$（当然在量子力学中，我们一般都取 $L^2(-\infty, +\infty)$。下面我们将尝试把它构建为一个矢量空间。设 $f$、$g$ 为希尔伯特空间中任取的两个函数：

- 定义矢量加法为 $(f + g)(x) = f(x) + g(x)$。
- 定义标量乘法为 $(cf)(x) = cf(x)$。

今后我们可能会将波函数写成矢量的形式，如 $\ket{\psi}$；但在定积分以及许多其它场景中，依然采用函数的形式。此外，希尔伯特空间也是一个内积空间，定义内积 $\braket{f}{g}$ 为：

$$
\begin{equation*}
    \braket{f}{g} = \int_a^b f^*(x)g(x)\,dx
\end{equation*}
$$

不难验证这个定义满足内积的要求：

$$
\begin{align*}
    \braket{g}{f} &= \int_a^b g^*(x)f(x)\,dx = \int_a^b (f^*(x)g(x))^*\,dx = \braket{f}{g}^* \\
    \braket{f}{f} &= \int_a^b f^*(x)f(x)\,dx = \int_a^b |f(x)|^2\,dx \ge 0\\
    \braket{af}{g + h} &= \int_a^b (af)^*(x)(g(x) + h(x))\,dx = \int_a^b a^*f^*(x)g(x) + \int_a^b a^*f^*(x)h(x) = a^*\braket{f}{g} + a^*\braket{f}{h}
\end{align*}
$$

现在，我们可以正式让波函数继承所有矢量的语言；比如两个正交的波函数定义为 $\braket{f}{g} = 0$，一集标准正交的函数 $f_n$ 需要满足 $\braket{f_i}{f_j} = \delta_{ij}$ 等等。特别地，如果一集函数能张成整个希尔伯特空间，我们称它们是 **完备（Complete）** 的，此时对于任意函数 $f(x)$ 都有：

$$
\begin{equation*}
    f(x) = \sum_{n=1}^\infty c_nf_n(x)
\end{equation*}
$$

为了找到这些系数 $c_n$，我们依然可以依照此前的公式（关于这个，有一个更加详细且复杂的证明过程，这里不展开了）：

$$
\begin{equation*}
    c_n = \braket{f_n}{f} = \int_a^b f_n^*(x)f(x)\,dx
\end{equation*}
$$

现在让我们考察物理量在线性代数中的含义。量子力学常将它们称为 **可观测量（Observables）**，其总是对应一个算符。上一节中我们尝试总结了一个规律，即可观测量的期望等于下面的积分：

$$
\begin{equation*}
    \exp{Q} = \int \Psi^*\hat{Q}\Psi\,dx = \braket{\Psi}{\hat{Q}\Psi}
\end{equation*}
$$

由于可观测量一定是实数，我们有 $\exp{Q} = \exp{Q}^*$，此时：

$$
\begin{equation*}
    \braket{\Psi}{\hat{Q}\Psi} = \exp{Q} = \exp{Q}^* = \braket{\hat{Q}\Psi}{\Psi}
\end{equation*}
$$

因此可观测量对应的算符一定是厄米特矩阵。算符自此会继承所有矩阵的术语和性质，我会用相同的方式处理两者（除了在显示写出矩阵的元素时）。

最后，我们还应将可观测量纳入到线性代数的语系中。注意到通常来讲，同样的状态 $\ket{\Psi}$ 下，每次测量的结果一般是不同的。在某些特殊状态，称为 **确定状态（Determinate State）** 下，或许我们能保证测量结果的稳定性。这实际上需要可观测量的标准差为零，即：

$$
\begin{equation*}
    \sigma^2 = \exp{(Q - \exp{Q})^2} = \braket{\Psi}{(\hat{Q} - \exp{Q})^2\Psi} = \braket{(\hat{Q} - \exp{Q})\Psi}{(\hat{Q} - \exp{Q})\Psi} = 0
\end{equation*}
$$

因此：

$$
\begin{equation*}
    \hat{Q}\ket{\Psi} = \exp{Q}\ket{\Psi}
\end{equation*}
$$

所以可观测量的期望值是其对应算符的特征值，而它的确定状态是这些特征值对应的特征矢量。

### 量子力学的统计解释

有了之前的所有铺垫，我们可以用正规的语言描述量子力学的统计意义了：在 $\Psi(x, t)$ 状态下测量粒子的可观测量 $Q(x, p)$ 时，我们一定会得到厄米特算符 $\hat{Q}$ 的一个特征值。对于离散波谱（即可数个特征值），每个特征值 $q_n$ 及特征矢量 $\ket{f_n}$ 出现的概率为 $|\braket{f_n}{\Psi}|^2$。这个过程称为 **波函数坍缩（Wave Function Collapse）**。

考虑两个可观测量 $A$ 与 $B$，它们的标准差满足：

$$
\begin{align*}
    \sigma_A^2 = \braket{(\hat{A} - \exp{A})\Psi}{(\hat{A} - \exp{A})\Psi} \\
    \sigma_B^2 = \braket{(\hat{B} - \exp{B})\Psi}{(\hat{B} - \exp{B})\Psi}
\end{align*}
$$

回忆施瓦兹不等式 $\sbraket{f}\sbraket{g} \ge |\braket{f}{g}|^2$，因此：

$$
\begin{equation*}
    \sigma_A^2\sigma_B^2
        \ge \left|\braket{(\hat{A} - \exp{A})\Psi}{(\hat{B} - \exp{B})\Psi}\right|^2 
\end{equation*}
$$

令 $z = \braket{(\hat{A} - \exp{A})\Psi}{(\hat{B} - \exp{B})\Psi)}$，复数拥有下面的性质：

$$
\begin{equation*}
    |z|^2 = [\text{Re}(z)]^2 + [\text{Im}(z)]^2 \ge [\text{Im}(z)]^2 = \left[\frac{1}{2i}(z - z^*)\right]^2
\end{equation*}
$$

同时：

$$
\begin{align*}
    z
        &= \braket{\Psi}{(\hat{A} - \exp{A})(\hat{B} - \exp{B})\Psi} \\
        &= \braket{\Psi}{\hat{A}\hat{B}\Psi} - \exp{A}\braket{\Psi}{\hat{B}\Psi} - \exp{B}\braket{\Psi}{\hat{A}\Psi} + \exp{A}\exp{B}\sbraket{\Psi} \\
        &= \exp{\hat{A}\hat{B}} - \exp{A}\exp{B} - \exp{B}\exp{A} + \exp{A}\exp{B} \\
        &= \exp{\hat{A}\hat{B}} - \exp{A}\exp{B}
\end{align*}
$$

总结下来：

$$
\begin{align*}
    \sigma_A^2\sigma_B^2 
         &\ge \left[\frac{1}{2i}\left(\exp{\hat{A}\hat{B}} - \exp{A}\exp{B} - \exp{\hat{B}\hat{A}} + \exp{B}\exp{A}\right)\right]^2 \\
         &= \left[\frac{1}{2i}\exp{\hat{A}\hat{B} - \hat{B}\hat{A}}\right]^2
\end{align*}
$$

我们将其中的 $\hat{A}\hat{B} - \hat{B}\hat{A}$ 定义为算符 $\hat{A}$、$\hat{B}$ 的 **交换子（Commutator）**，记为：

$$
\begin{equation*}
    [\hat{A}, \hat{B}] = \hat{A}\hat{B} - \hat{B}\hat{A}
\end{equation*}
$$

最终得到：

$$
\begin{equation*}
    \sigma_A^2\sigma_B^2 \ge \left(\frac{1}{2i}[\hat{A}, \hat{B}]\right)^2
\end{equation*}
$$

这个结论告诉我们的是，某个对两个可观测量均为确定的状态存在当且仅当它们的算符 **互易（Commute）**，即 $[\hat{A}, \hat{B}] = 0$。事实上，多数的算符之间都不可互易，比如位置和动量：

$$
\begin{align*}
    [\hat{x}, \hat{p}] f(x) 
        &= x\cdot i\hbar\frac{\partial}{\partial x} f(x) - i\hbar\frac{\partial}{\partial x} xf(x) \\
        &= i\hbar xf'(x) - i\hbar f(x) - i\hbar xf'(x) \\
        &= i\hbar f(x)
\end{align*}
$$

这就得到了 **标准对易关系（Canonical Commutation Relation）**：

$$
\begin{equation*}
    [\hat{x}, \hat{p}] = i\hbar
\end{equation*}
$$

代入此前标准差的不等式后有：

$$
\begin{equation*}
    \sigma_x\sigma_p \ge \frac{\hbar}{2}
\end{equation*}
$$

这便是著名的 **测不准原理（Uncertainty Principle）**；如果将定义 $\Delta x = \sigma_x$、$\Delta p = \sigma_p$，上面的不等式也常常写作：

$$
\begin{equation*}
    \Delta x\Delta p \ge \frac{\hbar}{2}
\end{equation*}
$$

对于这类对易子不为零的可观测量，我们称其相互 **不兼容（Incompatible）**。它们不存在相同的特征矢量。注意到一个正规矩阵，其自身和转置共轭总是互易的。

此前我们谈论的都是静态情况（时间固定）时波函数的特征。在一个动态系统中，考察物理量 $Q$ 期望值的变化率：

$$
\begin{align*}
    \frac{d}{dt}\exp{Q} = \frac{d}{dt}\braket{\Psi}{\hat{Q}\Psi}
        &= \braket{\frac{\partial \Psi}{\partial t}}{\hat{Q}\Psi} + \left\langle\Psi\left|\frac{\partial \hat{Q}}{\partial t}\Psi\right.\right\rangle + \left\langle\Psi\left|\hat{Q}\frac{\partial \Psi}{\partial t}\right.\right\rangle \\
        &= \frac{i}{\hbar}\braket{\hat{H}\Psi}{\hat{Q}\Psi} - \frac{i}{\hbar}\braket{\Psi}{\hat{Q}\hat{H}\Psi} + \exp{\frac{\partial \hat{Q}}{\partial t}} \\
        &= \frac{i}{\hbar}\exp{\left[\hat{H}, \hat{Q}\right]} + \exp{\frac{\partial Q}{\partial t}}
\end{align*}
$$

这就是 **埃伦费斯特定理（Ehrenfest Theorem）**。绝大多数情况下，算符都是静态的，因此我们可以认为可观测量的期望变化率仅和它与哈密顿量的对易子有关；当可观测量的算符与哈密顿算符互易时，$\exp{Q}$ 变成了守恒量。如果我们取 $Q = p$，可以得到另一个常用的公式：

$$
\begin{align*}
    \frac{d}{dt}\exp{p}
        &= \frac{i}{\hbar}\exp{\left[\hat{H}, \hat{p}\right]} \\
        &= \frac{i}{\hbar}\exp{\left[\hat{V}, \hat{p}\right]} \tag{展开哈密顿量，且 $\hat{p}$ 和 $\hat{p}^2$ 显然互易} \\
        &= \frac{i}{\hbar}\left[ \int\Psi^*V\frac{\partial\Psi}{\partial x}\,dx - \int\Psi^*\frac{\partial}{\partial x}\left(V\Psi\right)\,dx \right] \\
        &= \frac{i}{\hbar}\left[ \int\Psi^*V\frac{\partial\Psi}{\partial x}\,dx - \int\Psi^*\frac{\partial V}{\partial x}\Psi\,dx - \int\Psi^*V\frac{\partial\Psi}{\partial x}\,dx \right] \\
        &= -\frac{i}{\hbar}\exp{\frac{\partial V}{\partial x}}
\end{align*}
$$

有时候这个公式被直接称为埃伦费斯特定理，可以看到它和经典力学中“力”的联系：

$$
\begin{equation*}
    \frac{dp}{dt} = F = -\frac{dV}{dx}
\end{equation*}
$$

此外，如果令 $Q = x$，观察下面的等式：

$$
\begin{align*}
    \frac{d}{dt}\exp{x}
        &= \frac{i}{\hbar}\exp{\left[\hat{H}, \hat{x}\right]} \\
        &= \frac{i}{\hbar}\exp{\left[\frac{\hat{p}^2}{2m}, \hat{x}\right]} \tag{展开哈密顿量，且 $\hat{x}$ 和 $V$ 互易} \\
        &= \frac{i}{2m\hbar}\exp{\hat{p}[\hat{p}, \hat{x}] + [\hat{p}, \hat{x}]\hat{p}} \\
        &= \frac{i}{2m\hbar}\exp{\hat{p}\cdot2i\hbar} \\
        &= \frac{1}{m}\exp{p}
\end{align*}
$$

这个公式和经典力学中动量的定义对等：

$$
\begin{equation*}
    \frac{dx}{dt} = v = \frac{p}{m}
\end{equation*}
$$

最后，回忆测不准原理的一般形式：$\sigma_A^2\sigma_B^2 \ge \left(\dfrac{1}{2i}\exp{[\hat{A}, \hat{B}]}\right)^2$，令 $A = H$ 且 $B = Q$，我们有：

$$
\begin{align*}
    \sigma_H^2\sigma_Q^2
        &\ge \left(\frac{1}{2i}\exp{[\hat{H}, \hat{Q}]}\right)^2 \\
        &= \left(\frac{1}{2i}\frac{\hbar}{i}\frac{d\exp{Q}}{dt}\right)^2 \\
        &= \left(\frac{\hbar}{2}\right)^2\left(\frac{d\exp{Q}}{dt}\right)^2
\end{align*}
$$

因此 $\sigma_H\sigma_Q \ge \dfrac{\hbar}{2}\left|\dfrac{d\exp{Q}}{dt}\right|$。如果将能量变化和时间变化分别定义为：

$$
\begin{equation*}
    \Delta E = \sigma_H \qquad
    \Delta t = \sigma_Q \frac{1}{\left|\frac{d\exp{Q}}{dt}\right|}
\end{equation*}
$$

（上面 $\Delta t$ 的含义可以理解为令 $Q$ 变化一个标准差所需的时间）这样我们就得到了：

$$
\begin{equation*}
    \Delta E\Delta t \ge \frac{\hbar}{2}
\end{equation*}
$$

注意到能量和时间的关系类似于动量和位置的关系。我们会在后面的小节中进一步了解它们的对应关系。

### 三维空间及多粒子系统

此前为了证明的简洁性，我们讨论的均是一维系统。不过这些定理同时可以应用到三维空间中，只不过一些细节需要修正。比如波函数应该是 $\Psi(\mathbf{r}, t)$ 的形式，且薛定谔方程的哈密顿量就需要考虑三个方向上的动量：

$$
\begin{equation*}
    \hat{p}_x = -i\hbar\frac{\partial}{\partial x} \qquad
    \hat{p}_y = -i\hbar\frac{\partial}{\partial y} \qquad
    \hat{p}_z = -i\hbar\frac{\partial}{\partial z}
\end{equation*}
$$

因此我们得到：

$$
\begin{equation*}
    i\hbar\frac{\partial\Psi}{\partial t} = -\frac{\hbar^2}{2m}\nabla^2\Psi + V\Psi
\end{equation*}
$$

其中 $\nabla^2$ 是 **拉普拉斯算符（Laplacian Operator）**，在笛卡尔坐标系中定义为：

$$
\begin{equation*}
    \nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}
\end{equation*}
$$

三维系统中的波函数需要保证在全空间归一化：

$$
\begin{equation*}
    \int|\psi|^2\,d^3\mathbf{r} = 1
\end{equation*}
$$

这里的 $d^3\mathbf{r}$ 是 $dx\,dy\,dz$ 的简写。

如果我们引入另一个粒子，那么波函数就应该是 $\Psi(\mathbf{r}_1, \mathbf{r}_2, t)$ 的形式，且薛定谔方程会变为：

$$
\begin{equation*}
    i\hbar\frac{\partial\Psi}{\partial t} = -\frac{\hbar^2}{2m}\nabla_1^2\Psi_1 - \frac{\hbar^2}{2m}\nabla_2^2\Psi_2 + V(\mathbf{r}_1, \mathbf{r}_2, t)
\end{equation*}
$$

归一化的等式是：

$$
\begin{equation*}
    \int|\Psi(\mathbf{r}_1, \mathbf{r}_2, t)|^2\,d^3\mathbf{r}_1\,d^3\mathbf{r}_2 = 1
\end{equation*}
$$

### 对称性

**对称性（Symmetry）** 指的是某个使系统保持不变的变换。用简单的几何图形来说，正方形有 $\frac{\pi}{2}$ 的旋转对称性。量子力学中，对称性特指使得系统的哈密顿量不变的变换。让我们首先介绍一些和对称性相关的算符：

记 **平移算符（Translation Operator）** $\hat{T}(a)$ 为将波函数沿着 $x$ 轴向右平移 $a$ 的变换：

$$
\begin{equation*}
    \hat{T}(a)\psi(x) = \psi(x - a)
\end{equation*}
$$

记 **宇称算符（Parity Operator）** $\hat{\Pi}$ 为将波函数沿着 $y$ 轴镜像对称的变换：

$$
\begin{equation*}
    \hat{\Pi}\psi(x) = \psi(-x)
\end{equation*}
$$

#### 平移算符

我们可以进一步处理平移算符的定义（这里假设 $\psi(x)$ 是无限可导的）：

$$
\begin{align*}
    \hat{T}(a)\psi(x)
        &= \psi(x - a) \\
        &= \sum_{n=0}^\infty \frac{(-a)^n}{n!}\frac{d^n}{dx^2}\psi(x) \\
        &= \sum_{n=0}^\infty \frac{1}{n!}\left(\frac{-ia}{\hbar}\hat{p}\right)^n\psi(x)
\end{align*}
$$

因此实际上：

$$
\begin{equation*}
    \hat{T}(a) = \text{exp}\left(-\frac{ia}{\hbar}\hat{p}\right)
\end{equation*}
$$

我们称动量是平移的 **生成元（Generator）**。

### 自旋

对于粒子来说，其角动量由 **轨道（Orbital）** 角动量和 **自旋（Spin）** 构成；前者是粒子关于某个中心旋转导致的，其通过 $\mathbf{L} = \mathbf{r}\times\mathbf{p}$ 计算，后者则通过 $\mathbf{S} = I\boldsymbol{\omega}$ 得到。关于自旋，我们常通过其 $x$、$y$、$z$ 方向的分量描述，即 $S_x$、$S_y$、$S_z$。它们满足下面的对易关系：

$$
\begin{equation*}
    [\hat{S}_x, \hat{S}_y] = i\hbar \hat{S}_z \quad
    [\hat{S}_y, S_z] = i\hbar \hat{S}_x \quad
    [\hat{S}_z, S_x] = i\hbar \hat{S}_y
\end{equation*}
$$

这些等式可以暂时看成公理。可以看到三个方向上的自旋之间相互不兼容；有趣的是，它们中的每一个都和自旋的平方互易：

$$
\begin{align*}
    [\hat{S}^2, \hat{S}_x]
        &= [\hat{S}_x^2, \hat{S}_x] + [\hat{S}_y^2, \hat{S}_x] + [\hat{S}_z^2, \hat{S}_x] \\
        &= 0 + \hat{S}_y[\hat{S}_y, \hat{S}_x] + [\hat{S}_y, \hat{S}_x]\hat{S}_y + S_z[\hat{S}_z, \hat{S}_x] + [\hat{S}_z, \hat{S}_x]\hat{S}_z \\
        &= \hat{S}_y(-i\hbar \hat{S}_z) + (-i\hbar \hat{S}_z)\hat{S}_y + \hat{S}_z(i\hbar \hat{S}_y) + (i\hbar \hat{S}_y)\hat{S}_z \\
        &= 0
\end{align*}
$$

其等价的另一种形式为：

$$
\begin{equation*}
    [S^2, \mathbf{S}] = 0
\end{equation*}
$$

假设某个 $S_z$ 和 $S^2$ 的共同特征矢量为 $f$，则有：

$$
\begin{equation*}
    S^2f = \lambda f \qquad S_zf = \mu f
\end{equation*}
$$

为了展现特征值的分布，让我们引入 **阶梯算符（Ladder Operator）** $S_\pm$，其定义为：

$$
\begin{equation*}
    S_\pm = S_x \pm iS_y
\end{equation*}
$$

不难验证 $[S_z, S_\pm] = \pm\hbar S_\pm$，而 $[S^2, S_\pm] = 0$、注意到 $L_\pm f$ 同时也是 $S^2$ 和 $S_z$ 的特征矢量，其对应的特征值和 $f$ 相同：

$$
\begin{equation*}
    S^2(S_\pm f) = S_\pm(S^2 f) = S_\pm(\lambda f) = \lambda (S_\pm f)
\end{equation*}
$$

对于 $S_z$ 则有：

$$
\begin{equation*}
    S_z(S_\pm f) = [S_z, S_\pm]f + S_\pm S_zf = \pm\hbar S_\pm f + S_\pm(\mu f) = (\mu \pm \hbar)(S_\pm f)
\end{equation*}
$$

可以看到，$S_\pm$ 作用于函数 $f$ 上时会将其特征值提升或降低 $\hbar$，这也是阶梯算符名称的由来。特别地，$S_+$ 称为 **上升算符（Raising Operator）**，而 $S_-$ 称为 **下降算符（Lowering Operator）**。这样的上升和下降并不是没有限制的，其最小不能低于零，最高不能高于总量 $S$。假设这个最高点的特征矢量为 $f_t$，其对应的特征值为 $\hbar s$，此时：

$$
\begin{equation*}
    S_zf_t = \hbar sf_t \quad S^2f_t = \lambda f_t \quad S_+f_t = 0
\end{equation*}
$$

观察 $S_\pm S_\mp$：

$$
\begin{align*}
    S_\pm S_\mp
        &= (S_x \pm iS_y)(S_x \mp iS_y) \\
        &= S_x^2 + S_y^2 \mp i[S_x, S_y] \\
        &= S^2 - S_z^2 \pm \hbar S_z
\end{align*}
$$

将其中的 $S^2$ 用其它部分表示，则有：

$$
\begin{align*}
    S^2f_t
        &= (S_-S_+ + S_z^2 + \hbar S_z)f_t \\
        &= S_-(S_+f_t) + S_z^2f_t + \hbar S_zf_t \\
        &= \hbar^2s(s + 1)f_t
\end{align*}
$$

因此 $\lambda$ 和 $s$ 的关系为：

$$
\begin{equation*}
    \lambda = \hbar^2s(s + 1)
\end{equation*}
$$

类似地，如果设最低点的特征矢量为 $f_b$，其对应的特征值为 $\hbar \bar{s}$，我们可以得到：

$$
\begin{equation*}
    \lambda = \hbar^2\bar{s}(\bar{s} + 1)
\end{equation*}
$$

由 $s(s + 1) = \bar{s}(\bar{s} - 1)$，唯一合理的情形是 $\bar{s} = -s$。因此，粒子自旋的平方以及其 $z$ 方向上自旋分量满足下面的式子：

$$
\begin{equation*}
    S^2f_s^m = \hbar s(s + 1)f_s^m \qquad
    S_zf_s^m = \hbar mf_s^m
\end{equation*}
$$

其中 $s$ 可以是任意的整数或半整数，如 $0$、$3/2$ 等；而 $m$ 是从 $-s$ 到 $s$ 的所有整数（$s$ 是整数时）或所有半整数（$s$ 是半整数时）。一个更加亲切的写法是：

$$
\begin{equation*}
    S^2\ket{s\ m} = \hbar s(s + 1)\ket{s\ m} \qquad 
    S_z\ket{s\ m} = \hbar m\ket{s\ m}
\end{equation*}
$$

当 $s=0$ 时，整个系统只有一个特征矢量，它没有什么特别的。一个重要的自旋数是 $1/2$，组成所有物质的常规粒子，如质子、中子、电子都是 $1/2$ 自旋的。此时有两个特征矢量，分别是 $\ket{\frac{1}{2}\ \frac{1}{2}}$ 和 $\ket{\frac{1}{2}\ -\frac{1}{2}}$。我们分别将其称为 **上自旋（Spin Up）** 和 **下自旋（Spin Down）**，在量子力学中用 $\ket{\uparrow}$ 和 $\ket{\downarrow}$ 表示。对于 $1/2$ 自旋粒子的任何状态，都可以通过上下自旋的线性组合表示：

$$
\begin{equation*}
    \ket{\chi} = \alpha\ket{\uparrow} + \beta\ket{\downarrow}
\end{equation*}
$$

其中 $|\alpha|^2 + |\beta|^2 = 1$。因此，$1/2$ 自旋的粒子处于一个二维的希尔伯特空间中。下面让我们尝试得到 $S^2$ 和 $S_z$ 的矩阵形式：

$$
\begin{align*}
    \left.
    \begin{split}
        \hat{S}^2\ket{\uparrow} = \frac{3}{4}\hbar^2\ket{\uparrow} \\
        \hat{S}^2\ket{\downarrow} = \frac{3}{4}\hbar^2\ket{\downarrow}
    \end{split}\right\}
    &\implies \hat{S}^2 = \frac{3}{4}\hbar^2\hat{I} \\
    \left.
    \begin{split}
        \hat{S}_z\ket{\uparrow} = \frac{\hbar}{2}\ket{\uparrow} \\
        \hat{S}_z\ket{\downarrow} = -\frac{\hbar}{2}\ket{\downarrow}
    \end{split}\right\}
    &\implies \hat{S}_z = \frac{\hbar}{2}\begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
\end{align*}
$$

经过一些计算后，我们也可以得到 $\hat{S}_x$ 和 $\hat{S}_y$ 的矩阵形式：

$$
\begin{equation*}
    \hat{S}_x = \frac{\hbar}{2}\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \quad
    \hat{S}_y = \frac{\hbar}{2}\begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}
\end{equation*}
$$

显然，$\hat{S}_z$ 的特征矢量是 $\ket{\uparrow}$ 与 $\ket{\downarrow}$，其对应的特征值是 $\pm\frac{\hbar}{2}$。对于 $\hat{S}_x$ 和 $\hat{S}_y$，其特征值应该与 $\hat{S}_z$ 的相同（毕竟最初 $z$ 就是任选的），但其对应的特征矢量有所不同。以 $\hat{S}_x$ 为例，其特征矢量是：

$$
\begin{equation*}
    \ket{\uparrow}^{(x)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix} \qquad
    \ket{\downarrow}^{(x)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}} \end{pmatrix}
\end{equation*}
$$

在 $\ket{\uparrow}^{(x)}$ 和 $\ket{\downarrow}^{(x)}$ 组成的基下，任意状态 $\ket{\chi}$ 可以写成：

$$
\begin{equation*}
    \ket{\chi} = \frac{\alpha + \beta}{\sqrt{2}}\ket{\uparrow}^{(x)} + \frac{\alpha - \beta}{\sqrt{2}}\ket{\downarrow}^{(x)}
\end{equation*}
$$

从 $z$ 方向的基到 $x$ 方向的基，变换矩阵为：

$$
\begin{equation*}
    \hat{H} = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}
\end{equation*}
$$

类似地，$y$ 方向的基是：

$$
\begin{equation*}
    \ket{\uparrow}^{(y)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}}i \end{pmatrix} \qquad
    \ket{\downarrow}^{(y)} = \begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}}i \end{pmatrix}
\end{equation*}
$$

任意矢量 $\ket{\chi}$ 可以写成：

$$
\begin{equation*}
    \ket{\chi} = \frac{\alpha + i\beta}{\sqrt{2}}\ket{\uparrow}^{(y)} + \frac{\alpha - i\beta}{\sqrt{2}}\ket{\downarrow}^{(y)}
\end{equation*}
$$

我们花了不小的篇幅介绍 $1/2$ 自旋粒子的这样一个两状态系统，这是因为它拥有和量子电路中的最小信息单元：量子比特完全相同的性质。事实上，利用电子自旋是制作量子计算机的一种可能性。下一章中，我们将正式进入量子电路的学习。

## 量子电路

### 基本概念

和传统电路相似，**量子电路（Quantum Circuit）** 是由多个 **量子比特（Qubit）** 和 **量子门（Quantum Gate）** 构成的。量子比特和传统计算机中的比特类似，其确定状态有 $\ket{0}$ 和 $\ket{1}$ 两种。不过其一般状态如下所示：

$$
\begin{equation*}
    \ket{\psi} = \alpha\ket{0} + \beta\ket{1}
\end{equation*}
$$

回忆量子力学中的知识，这里的 $\ket{\psi}$ 是希尔伯特空间任取的矢量，而 $\ket{0}$ 与 $\ket{1}$ 是一组特定的特征矢量。因此当我们对如上状态的量子比特测量时，它有 $|\alpha|^2$ 的概率展示 $\ket{0}$，而 $|\beta|^2$ 的概率展示 $\ket{1}$，同时满足：

$$
\begin{equation*}
    |\alpha|^2 + |\beta|^2 = 1
\end{equation*}
$$

我们可以让 $\ket{0}$ 和 $\ket{1}$ 作为基，此时下面的矢量唯一代表了一个状态：

$$
\begin{equation*}
    \ket{\psi} = \begin{pmatrix} \alpha \\ \beta \end{pmatrix}
\end{equation*}
$$

从形式来看，这和 $1/2$ 自旋粒子的 $z$ 方向自旋的两种状态完全一致。类似地，以 $x$ 方向旋量为基时，我们可以得到：

$$
\begin{equation*}
    \ket{\psi} = \frac{\alpha + \beta}{\sqrt{2}}\ket{+} + \frac{\alpha - \beta}{\sqrt{2}}\ket{-}
\end{equation*}
$$

这里的 $\ket{\pm}$ 就是上一章中给出的 $\ket{\uparrow}^{(x)}$ 与 $\ket{\downarrow}^{(x)}$。类似地，我们可以给出以 $y$ 方向旋量为基的形式：

$$
\begin{equation*}
    \ket{\psi} = \frac{\alpha + i\beta}{\sqrt{2}}\ket{i+} + \frac{\alpha - i\beta}{\sqrt{2}}\ket{i-}
\end{equation*}
$$

当存在多个量子比特时，我们将其直接写成 $\ket{\psi_1\dots\psi_n}$ 的形式。注意到这是克罗内克积 $\ket{\psi_1}\otimes\dots\otimes\ket{\psi_n}$ 的简写。对于以 $\ket{0}$ 和 $\ket{1}$ 为基的矢量，观察到它的形式是：

$$
\begin{equation*}
    \ket{\psi} = \ket{b_{n-1}\dots b_0} \qquad b_j \in \{0, 1\}
\end{equation*}
$$

此时我们可以将其理解为二进制整数（左侧是最大位），然后将状态转写成：

$$
\begin{equation*}
    \ket{\sum_{i=0}^{n-1}2^i} = \ket{b_{n-1}\dots b_0}
\end{equation*}
$$

因此后面如果看到形如 $\ket{5}$ 的状态，请将其理解成 $|\underbrace{0\dots0}_{n-3}110\rangle$。

量子门是接受一组量子比特输入，并输出相同数量量子比特的装置。这点和传统电路有明显的不同：后者中的许多电路门，如异或，输入两个比特但只返回一个比特。同时，我们要求所有的量子门都是幺正矩阵。这两个性质使得所有量子门都是可逆的：对于任意的输入 $\ket{\psi_{n-1}\dots\psi_0}$ 和量子门 $U$，其输出结果 $\ket{\psi_{n-1}'\dots\psi_0'} = U\ket{\psi_{n-1}\dots\psi_0}$ 可以再次输入同样的量子门得到：

$$
\begin{equation*}
    U\ket{\psi_{n-1}'\dots\psi_0'} = UU\ket{\psi_{n-1}\dots\psi_0} = \ket{\psi_{n-1}\dots\psi_0}
\end{equation*}
$$

由于量子比特可以理解为矢量，量子门成为和矩阵等价的量；因此我会将量子门（以及其矩阵形式）统一写成大写的字母。

下面的几个小节中，我们将介绍一些常见的量子门及其运算性质。在此之前，我们还需要明确 **测量（Mesaurement）** 这个概念。如上一章节所属，对量子系统进行测量时，系统的波函数会坍缩为一个特征态；这个过程也可以利用矩阵描述。我们将所有量子测量用一系列 **测量算符（Measurement Operator）** 的集合 $\{M_m\}$，其中 $m$ 是一个测量结果。得到结果 $m$ 的概率可以通过下面的公式计算：

$$
\begin{equation*}
    p(m) = \bracket{\psi}{M_m^\dagger M_m}{\psi}
\end{equation*}
$$

在测量后，系统的状态是（坍缩态）：

$$
\begin{equation*}
    \frac{M_m\ket{\psi}}{\sqrt{\bracket{\psi}{M_m^\dagger M_m}{\psi}}}
\end{equation*}
$$

一个很好理解的例子是投影算符，如 $\ket{0}\bra{0}$。它代表了 $\ket{0}$ 的测量算符。概率公式此时变成了熟悉的形式：

$$
\begin{equation*}
    p(m) = \bra{\psi} (\ket{0}\bra{0})\ket{\psi} = |\braket{0}{\psi}|^2
\end{equation*}
$$

而测量之后的状态是：

$$
\begin{equation*}
    \frac{(\ket{0}\bra{0})\ket{\psi}}{\sqrt{|\braket{0}{\psi}|^2}} = \frac{\braket{0}{\psi}}{|\braket{0}{\psi}|}\ket{0}
\end{equation*}
$$

### 常见的量子门

#### 泡利门

**泡利门（The Pauli Gates）** 是自旋为 $1/2$ 的粒子的 $x$、$y$、$z$ 三方向自选分量对应的算符，在量子计算中记为 $X$、$Y$ 和 $Z$。它们都接受一个量子比特，映射规则如下：

$$
\begin{equation*}
    X: 
    \begin{cases}
        \ket{0} \mapsto \ket{1} \\
        \ket{1} \mapsto \ket{0}
    \end{cases} \quad
    Y:
    \begin{cases}
        \ket{0} \mapsto -i\ket{1} \\
        \ket{1} \mapsto i\ket{0}
    \end{cases} \quad
    Z:
    \begin{cases}
        \ket{0} \mapsto \ket{0} \\
        \ket{1} \mapsto -\ket{1}
    \end{cases}
\end{equation*}
$$

它们的矩阵形式可以通过矩阵元素的提取公式 $T_{ij} = \bracket{e_i}{T}{e_j}$ 得到，以 $X$ 为例：

$$
\begin{equation*}
    X = \begin{pmatrix}
        \bracket{0}{X}{0} & \bracket{0}{X}{1} \\
        \bracket{1}{X}{0} & \bracket{1}{X}{1}
    \end{pmatrix} = \begin{pmatrix}
        \braket{0}{1} & \braket{0}{0} \\
        \braket{1}{1} & \braket{1}{0}
    \end{pmatrix} = \begin{pmatrix}
        0 & 1 \\ 1 & 0
    \end{pmatrix}
\end{equation*}
$$

类似地我们得到：

$$
\begin{equation*}
    Y = \begin{pmatrix}
        0 & -i \\ i & 0
    \end{pmatrix} \qquad
    Z = \begin{pmatrix}
        1 & 0 \\ 0 & -1
    \end{pmatrix}
\end{equation*}
$$

泡利门之间有着有趣的联系（它们是 **泡利群（Pauli Group）** 的生成元），值得一提的有：

- 每一个门都可以表示为其它两个的右手积（考虑笛卡尔系中的基矢量的叉积关系，如 $\hat{z} = \hat{x}\times\hat{y}$）乘以 $i$，即：
$$
\begin{equation*}
    X = iYZ \quad
    Y = iZX \quad
    Z = iXY
\end{equation*}
$$
- 任两个门满足反对称性：
$$
\begin{equation*}
    YX = -XY \quad
    ZY = -YZ \quad
    XZ = -ZX
\end{equation*}
$$

从基本观察中可以发现，$X$ 将 $\ket{0}$ 和 $\ket{1}$ 的概率调转，其行为和传统电路中的 $NOT$ 比较相似，因此也被称为 **非门（Not Gate）**。此外的两个门目前看起来还比较抽象，我们会在合适的实际详细介绍它们的几何意义。

#### 相位偏移门

**相位偏移门（Phase Shift Game）** 对 $\ket{1}$ 进行 $\phi$ 角度的相位偏移，其映射规则如下：

$$
\begin{equation*}
    P(\phi):
    \begin{cases}
        \ket{0} \mapsto \ket{0} \\
        \ket{1} \mapsto e^{i\phi}\ket{1}
    \end{cases}
\end{equation*}
$$

因此它的矩阵形式为：

$$
\begin{equation*}
    P(\phi) = \begin{pmatrix}
        1 & 0 \\ 0 & e^{i\phi}
    \end{pmatrix}
\end{equation*}
$$

相位偏移并不对原状态的概率进行改变，而仅仅改变 $\ket{1}$ 分量上的相位。当 $\phi$ 取特定值时，我们常常使用特殊的记号。比如：

- $\phi = 0$，此时正好是单位门 $I$。
- $\phi = \frac{\pi}{4}$，此时记作 $T$。有趣的是，由于历史原因，它也被称为 $\frac{\pi}{8}$ 门。
- $\phi = \frac{\pi}{2}$，此时记作 $S$。
- $\phi = \pi$，此时正好是泡利门 $Z$。

相位偏移门是斜厄米特矩阵，而非厄米特矩阵，因此它们的共轭转置和其自身不同。

#### 旋转门

通过泡利门我们可以定义与之相关的 **旋转门（Rotation Gate）**：

$$
\begin{equation*}
    R_x(\theta) = e^{-i\theta X/2} \qquad
    R_y(\theta) = e^{-i\theta Y/2} \qquad
    R_z(\theta) = e^{-i\theta Z/2}
\end{equation*}
$$

回忆幺正厄米特矩阵的运算性质（$U^2 = I$），它们的矩阵形式可以计算得到：

$$
\begin{align*}
    R_x(\theta) &= \begin{pmatrix}
        \cos\frac{\theta}{2} & -i\sin\frac{\theta}{2} \\
        -i\sin\frac{\theta}{2} & \cos\frac{\theta}{2}
    \end{pmatrix} &
    R_y(\theta) &= \begin{pmatrix}
        \cos\frac{\theta}{2} & -\sin\frac{\theta}{2} \\
        \sin\frac{\theta}{2} & \cos\frac{\theta}{2}
    \end{pmatrix} &
    R_z(\theta) &= \begin{pmatrix}
        e^{-i\theta/2} & 0 \\
        0 & e^{i\theta/2}
    \end{pmatrix}
\end{align*}
$$


#### 哈达玛门

**哈达玛门（Hadamard Gate）** 将量子比特转变为一个叠加状态：

$$
\begin{equation*}
    H: 
    \begin{cases}
        \ket{0} \mapsto \dfrac{\ket{0} + \ket{1}}{\sqrt{2}} = \ket{+} \\
        \ket{1} \mapsto \dfrac{\ket{0} - \ket{1}}{\sqrt{2}} = \ket{-}
    \end{cases}
\end{equation*}
$$

其矩阵形式是：

$$
\begin{equation*}
    H = \frac{1}{\sqrt{2}}\begin{pmatrix}
        1 & 1 \\ 1 & -1
    \end{pmatrix}
\end{equation*}
$$

多数情况下，量子比特的初始状态都是 $\ket{0}$，此时我们可以通过哈达玛门将其变成 $\ket{0}$ 和 $\ket{1}$ 的叠加态，便于后续的操作。

前面我们使用的 $\ket{+}$ 和 $\ket{-}$ 构成了另一组常用的基。有趣的是，我们可以将它们通过哈达玛门再次转换成 $\ket{0}$ 和 $\ket{1}$ 的基：

$$
\begin{equation*}
    H\ket{+} = \ket{0} \quad
    H\ket{-} = \ket{1}
\end{equation*}
$$

我们将此前介绍的 $\ket{0}$ 和 $\ket{1}$（这样通过数字表示的）基称为 **计算基（Computational Basis）**，其实际相当于空间中笛卡尔系下的标准正交基。由 $\ket{+}$ 和 $\ket{-}$ 构成的基则称为 **哈达玛基（Hadamard Basis）**。今后这两种基都会被频繁用到。

#### 受控非门

**受控门（Controlled Gate）** 接收两个量子比特，其中一个是 **控制量子比特（Controlled Qubit）**，当且仅当该比特中含有 $\ket{1}$ 分量时，会等概率向另一个比特执行特定操作。其中最常用的便是 **受控非门（Controlled Not Gate）**，由于非门就是泡利 $X$ 门，我们将其记作 $C_X$（更常见的记法是 $CX$ 或 $CNOT$，但为了公式的紧凑，我会在笔记中采用这种形式）：

$$
\begin{equation*}
    C_X:
    \begin{cases}
        \ket{00} \mapsto \ket{00} \\
        \ket{01} \mapsto \ket{01} \\
        \ket{10} \mapsto \ket{11} \\
        \ket{11} \mapsto \ket{10}
    \end{cases}
\end{equation*}
$$

其矩阵形式为：

$$
\begin{equation*}
    C_X = \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 1 & 0
    \end{pmatrix}
\end{equation*}
$$

观察上面的规律，我们不难写出任意单比特量子门的受控门形式：

$$
\begin{equation*}
    C_U = \begin{pmatrix}
        I & 0 \\
        0 & U
    \end{pmatrix}
\end{equation*}
$$

举例来说，受控 $Z$ 门的矩阵形式是：

$$
\begin{equation*}
    C_Z = \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & -1
    \end{pmatrix}
\end{equation*}
$$

值得注意的是，$CNOT$ 门从效果上来看和传统电路门中的异或门 $XOR$ 效果完全相同（第二个量子比特是 $XOR$ 的结果），我们在后面可能会利用到这一点。

为了让 $C_X$ 能作用于哈达玛基，我们需要在前后对两个量子比特都使用 $H$ 进行变换，即：

$$
\begin{align*}
    {C}_{X}^{(\mathscr{B}_H)} 
        &= (H\otimes H)C_X(H\otimes H) \\
        &= \begin{pmatrix}
            1 & 1 & 1 & 1 \\
            1 & -1 & 1 & -1 \\
            1 & 1 & -1 & -1 \\
            1 & -1 & -1 & 1
        \end{pmatrix}
        \begin{pmatrix}
            1 & 0 & 0 & 0 \\
            0 & 1 & 0 & 0 \\
            0 & 0 & 0 & 1 \\
            0 & 0 & 1 & 0
        \end{pmatrix}
        \begin{pmatrix}
            1 & 1 & 1 & 1 \\
            1 & -1 & 1 & -1 \\
            1 & 1 & -1 & -1 \\
            1 & -1 & -1 & 1
        \end{pmatrix} \\
        &= \begin{pmatrix}
            0 & 1 & 0 & 0 \\
            1 & 0 & 0 & 0 \\
            0 & 0 & 1 & 0 \\
            0 & 0 & 0 & 1
        \end{pmatrix}
\end{align*}
$$

有趣的是，这恰好是 $C_X$ 对两个量子比特作用颠倒的结果。换句话说，在哈达玛基中，下面的变换成立：

$$
\begin{equation*}
    C_X^{(\mathscr{B}_H)}:
    \begin{cases}
        \ket{++} \mapsto \ket{++} \\
        \ket{-+} \mapsto \ket{-+} \\
        \ket{+-} \mapsto \ket{--} \\
        \ket{--} \mapsto \ket{+-}
    \end{cases}
\end{equation*}
$$

### 密度算符

本章初，我们提到量子电路可以通过一个矢量（如两状态系统 $\ket{\psi} = \alpha\ket{0} + \beta\ket{1}$）表示。本节中让我们介绍一种更加通用的表示量子系统状态的方式：**密度算符（Density Operator）**，它可以用于描述多个量子系统混合而成的复杂系统，称为 **系综（Ensemble）**。当系综只存在一种状态时，我们称这样的系统处于 **纯态（Pure State）**，此时定义密度算符为 $\rho = \ket{\psi}\bra{\psi}$，这也是我们此前一直讨论的情况。回忆此前对算符期望值的定义 $\exp{A} = \bracket{\psi}{A}{\psi}$，因此：

$$
\begin{align*}
    \exp{A} = \bracket{\psi}{A}{\psi} 
        &= \sum_{i,j=1}^n c_i^*c_j \bracket{\psi_i}{A}{\psi_j} \\
        &= \sum_{i,j=1}^n \braket{\psi_j}{\psi}\braket{\psi}{\psi_i} \bracket{\psi_i}{A}{\psi_j} \\
        &= \sum_{i,j=1}^n \bracket{\psi_j}{\rho}{\psi_i} \bracket{\psi_i}{A}{\psi_j} \\
        &= \sum_{j=1}^n \bra{\psi_j} \rho\left(\sum_{i=1}^n \ket{\psi_i}\bra{\psi_i}\right) A\ket{\psi_j} \\
        &= \sum_{j=1}^n \bracket{\psi_j}{\rho A}{\psi_j} \\
        &= \text{tr}(\rho A)
\end{align*}
$$

这是一个重要的结论。

现在让我们考虑一个由两种状态的系综：

$$
\begin{equation*}
    \ket{\psi} = \alpha_1\ket{\mu} + \alpha_2\ket{\nu} \qquad
    \ket{\phi} = \beta_1\ket{\mu} + \beta_2\ket{\nu}
\end{equation*}
$$

这两个状态分别的密度算符记为：

$$
\begin{equation*}
    \rho_\psi = \ket{\psi}\bra{\psi} \qquad
    \rho_\phi = \ket{\phi}\bra{\phi}
\end{equation*}
$$

这个系综的密度算符则定义为两者的加权和：

$$
\begin{equation*}
    \rho = p\rho_\psi + (1-p)\rho_\phi
\end{equation*}
$$

其余情况下便是 **混合态（Mixed State）**。一些语境下，我们会用混合态指代所有量子系统可能的状态（也就是包括纯态），而纯态指的是此前我们用的状态矢量 $\ket{\psi}$，与密度算符相对。

$$
\begin{equation*}
    \rho \equiv \sum_{i} p_i\ket{\psi_i}\bra{\psi_i}
\end{equation*}
$$

如果记密度算符中每一种概率对应的状态为 $\rho_i$，则显然有 $\rho = \sum_i p_i\rho_i$。

密度算符总是满足下面的三个性质：

- $\rho$ 是厄米特算符。
- $\text{tr}(\rho) = 1$。证明如下：

$$
\begin{equation*}
    \text{tr}(\rho) = \text{tr}\left(\sum_{i=1}^n p_i\ket{\psi_i}\bra{\psi_i}\right) = \sum_{i=1}^n p_i\text{tr}(\ket{\psi_i}\bra{\psi_i}) = \sum_{i=1}^n p_i \braket{\psi_i}{\psi_i} = \sum_{i=1}^n p_i = 1
\end{equation*}
$$

- $\bracket{\psi}{\rho}{\psi} \ge 0$，即 $\rho$ 是正算符。证明如下：

$$
\begin{equation*}
    \bracket{\psi}{\rho}{\psi} = \sum_{i=1}^n p_i \braket{\psi}{\psi_i}\braket{\psi_i}{\psi} = \sum_{i=1}^n p_i |\braket{\psi}{\psi_i}|^2 \ge 0
\end{equation*}
$$


## 量子算法

### 量子傅里叶变换

**量子傅里叶变换（Quantum Fourier Transform, QFT）** 是在量子计算中常用的变换，它将 $N$ 维空间中的任一个矢量变换为特定的一组基的线性组合。首先对于基 $\ket{j}$，其变换定义如下：

$$
\begin{equation*}
    \ket{j} = \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1} e^{2\pi ijk/N}\ket{k}
\end{equation*}
$$

对于任意矢量，则相当于进行了如下变换：

$$
\begin{equation*}
    \sum_{j=0}^{N-1} x_j\ket{j} \mapsto \sum_{k=0}^{N-1} y_k\ket{k}
\end{equation*}
$$

量子计算中，我们选取 $N = 2^n$，即一个 $n$ 量子比特的系统。此时：

$$
\begin{align*}
    \ket{j} 
        =\ & \ket{j_1\dots j_n} \\
        \mapsto\ & \frac{1}{2^{n/2}} \sum_{k=0}^{2^n-1} e^{2\pi ijk/2^n} \ket{k} \\
        =\ & \frac{1}{2^{n/2}} \sum_{k_1=0, \dots k_n=0}^1 e^{2\pi ij\sum_{l=1}^nk_l2^{-l}} \ket{k_1\dots k_n} \\
        =\ & \frac{1}{2^{n/2}} \sum_{k_1=0, \dots k_n=0}^1 \bigotimes_{l=1}^n e^{2\pi ijk_l2^{-l}}\ket{k_l} \\
        =\ & \frac{1}{2^{n/2}} \bigotimes_{l=1}^n\left(\ket{0} + e^{2\pi ij2^{-l}}\ket{1}\right) \\
        =\ & \frac{\left(\ket{0} + e^{2\pi i0.j_1\dots j_n}\ket{1}\right)\otimes\dots\otimes\left(\ket{0} + e^{2\pi i0.j_n}\ket{1}\right)}{2^{n/2}}
\end{align*}
$$

至此我们成功将这个转换化成了对每个量子比特单独的转换。为了方便表示这个电路，让我们记：

$$
\begin{equation*}
    R_k = \begin{pmatrix}
        1 & 0 \\ 0 & e^{2\pi i/2^k} 
    \end{pmatrix} = P(2^{1-k}\pi)
\end{equation*}
$$

因此不难看出，对于 $\ket{j_m}$，我们只需对其施加一个哈达玛门（从而生成 $\ket{0}$ 和 $\ket{1}$ 的叠加态），然后从 $R_2$ 一直操作到 $R_{n+1-m}$ 即可得到 $\ket{k_{n+1-m}}$，我们需要让它和 $\ket{j_{n+1-m}}$ 得到的结果互换。同时由于有 $n$ 个量子比特，$n$ 个哈达玛门的系数正好对应 $2^{n/2}$，至此我们就完成了电路。由于每一个组成部分都是幺正的，量子傅里叶变换也是幺正的。

现在考量这个算法的时间复杂度。我们一共使用了 $\frac{n(n+1)}{2} + n/2 \approx n^2$ 个量子门，因此算法的时间复杂度是 $O(n^2)$。

### 相位估计

我们知道，幺正矩阵的特征值在复平面的单位圆上。因此我们可以将这个特征值记为 $e^{2\pi i\varphi}$，其中 $\varphi \in [0, 1]$（这是一个很重要的设置）。**相位估计（Phase Estimation）** 就是估计 $\varphi$ 的算法。在这个算法中电路分为两个寄存器部分，其一是拥有 $t$ 个量子比特的，初始值为 $\ket{0\dots 0}$ 的寄存器，$t$ 的大小决定了估计的精准度；另一个是拥有 $n$ 个量子比特的，用于让 $\varphi$ 对应的特征矢量 $\ket{u}$ 通过的寄存器。这里的 $\ket{u}$ 我们是通过某个 **神谕（Oracle）** 准备得到的状态。

一切准备就绪，让我们对第一个寄存器的每一个量子比特进行哈达玛操作，随后从最低到最高位，设置 $j \in [0, t-1]$ 对第二个寄存器的受控 $U^{2^j}$ 门。我们最后的结果是：

$$
\begin{align*}
    |\underbrace{0\dots0}_t\rangle \quad
        \mapsto \quad & \frac{1}{\sqrt{2}}\left(\ket{0} + e^{2\pi i2^{t-1}\varphi}\ket{1}\right)\otimes\dots\otimes\frac{1}{\sqrt{2}}\left(\ket{0} + e^{2\pi i2^0\varphi}\ket{1}\right) \\
        = \quad & \frac{1}{2^{t/2}}\sum_{k=0}^{2^t-1} e^{2\pi i\varphi k}\ket{k}
\end{align*}
$$

至于第二个寄存器，由于输入是特征矢量 $\ket{u}$，它的输出依然是 $\ket{u}$，因此上面我们将其省略了。

假设 $\varphi \approx 0.\varphi_1\dots\varphi_t$，那么上面得到的结果相当于：

$$
\begin{equation*}
    \frac{1}{2^{t/2}}\left(\ket{0} + e^{2\pi i0.\varphi_t}\ket{1}\right)\dots\left(\ket{0} + e^{2\pi i0.\varphi_1\dots\varphi_t}\ket{1}\right)
\end{equation*}
$$

注意到这正是量子傅里叶变换从 $\ket{\varphi_1\dots\varphi_t}$ 映射得到结果的上下颠倒！因此只需进行一次 **逆量子傅里叶变换（Inverse Quantum Fourier Transform, IQFT）** 我们就可以得到 $\varphi$ 前 $t$ 个小数位的精确值。幸运的是，量子门的逆变换非常容易得到，因为幺正矩阵的逆便是它的共轭转置。

下面让我们计算这个算法的误差。对于 $t$ 个量子比特的寄存器，假设 $b \in [0, 2^{t-1}]$ 令 $b/2^t = 0.b_1\dots d_t$ 是比 $\varphi$ 小的最精确估计，此时经历 IQFT 可以得到：

$$
\begin{equation*}
    \frac{1}{2^t}\sum_{k, l=0}^{2^t-1} e^{-2\pi ikl/2^t}e^{2\pi i\varphi k}\ket{l}
\end{equation*}
$$

记 $\ket{(b+l)\ \text{mod}\ 2^t} = \alpha_l$，则：

$$
\begin{align*}
    \alpha_l
        &= \frac{1}{2^t}\sum_{k=0}^{2^t-1} \left(e^{2\pi i[\varphi - (b+l)/2^t]}\right)^k \tag{注意这是一个几何级数} \\
        &= \frac{1}{2^t}\frac{1 - e^{2\pi i[2^t\varphi - (b+l)]}}{1 - e^{2\pi i[\varphi - (b+1)/2^t]}} \\
        &= \frac{1}{2^t}\frac{1 - e^{2\pi i(2^t\delta - l)}}{1 - e^{2\pi i(\delta - l/2^t)}} \tag{令 $\delta = \varphi - b/2^t$}
\end{align*}
$$

现在对于测量结果 $m$，为了让它和最精确估计 $b$ 的误差不超过 $e_0 \in \mathbb{Z}^+$，我们可以计算 $\text{Pr}(|m-b|>e_0)$，即：

$$
\begin{equation*}
    \text{Pr}(|m-b|>e_0) = \sum_{-2^{t-1}<l\le-(e_0+1)} |\alpha_l|^2 + \sum_{e_0+1\le l\le 2^{t-1}} |\alpha_l|^2
\end{equation*}
$$

考虑到 $\delta - l/2^t \in [-1/2, 1/2]$（因为 $\delta \approx 0$ 且 $l/2^t \in [-1/2, 1/2]$），因此 $2\pi i(\delta - l/2^t) \in [-\pi i, \pi i]$。此时 $|4(\delta - l/2^t)|/\pi$ 是 $|1 - \text{exp}[2\pi i(\delta - l/2^t)]|$ 的一个下界。此外，我们总可以取 $2$ 作为 $|1 - \text{exp}[2\pi i(2^t - \delta - l)]|$ 的上界，因此：

$$
\begin{equation*}
    |\alpha_l| \le \frac{1}{2^{t+1}(\delta - l/2^t)}
\end{equation*}
$$

代入到前面的概率等式中，便有：

$$
\begin{align*}
    \text{Pr}(|m-b|>e_0)
        &\le \frac{1}{4} \left[\sum_{l=-2^{t-1}+1}^{-e_0-1} \frac{1}{(l-2^t\delta)^2} + \sum_{l=e_0+1}^{2^{t-1}} \frac{1}{(l-2^t\delta)^2}\right] \\
        &\le \frac{1}{4} \left[\sum_{l=-2^{t-1}+1}^{-e_0-1} \frac{1}{l^2} + \sum_{l=e_0+1}^{2^t-1} \frac{1}{(l-1)^2}\right] \\
        &\le \frac{1}{2} \sum_{t=e_0}^{2^{t-1}-1}\frac{1}{l^2} \\
        &\le \frac{1}{2} \int_{e_0-1}^{2^{t-1}-1} \frac{1}{l^2}\,dl \\
        &= \frac{1}{2(e_0-1)}
\end{align*}
$$

因此如果想让 $\varphi$ 的结果有至少 $2^{-n}$ 的精确度，此时取 $e_0 \ge 2^{t-n}-1$，我们得到：

$$
\begin{equation*}
    t \ge n + \left\lceil\log\left(2 + \frac{1}{\epsilon}\right)\right\rceil
\end{equation*}
$$

其中 $\epsilon$ 是无法达到这个精确度的概率。

相位估计的一个前提条件是我们能找到“神谕”得到特征矢量 $\ket{u}$，如果我们找不到怎么办呢？假设我们提供的是任意的矢量 $\ket{\psi}$，它可以表示成一组特征矢量的线性组合：

$$
\begin{equation*}
    \ket{\psi} = \sum_{k=1}^nc_k\ket{u_k}
\end{equation*}
$$

将上面的内容输入算法，我们会得到：

$$
\begin{equation*}
    \sum_{k=1}^nc_k\ket{\tilde{\varphi}_k}\ket{u_k}
\end{equation*}
$$

其中 $\tilde{\varphi}_k$ 是 $\ket{u_k}$ 对应的特征值估计，有 $|c_k|^2$ 的概率被测量得到。因此尽管我们输入的是任意的矢量，依然能得到某个特征值的估计。

### 寻阶算法

这里首先进行代数相关的铺垫。对于正整数 $x$，其模一个互素正整数 $N > x$ 的 **阶（Order）** 是最小的满足下面等式的正整数 $r$：

$$
\begin{equation*}
    x^r = 1\ (\text{mod}\ N)
\end{equation*}
$$

举例来说，对于 $N=7$，$2$ 的阶是 $3$（因为 $2^3 = 8 = 1\ (\text{mod}\ 7)$），$3$ 的阶是 $6$（因为 $3^6 = 729 = 1\ (\text{mod}\ 7)$）。

寻阶算法就是对任意给定的 $x, N$，找到 $x$ 模 $N$ 的阶。这个算法对应的变换常写成：

$$
\begin{equation*}
    U\ket{y} = \begin{cases} 
        \ket{xy\ (\text{mod}\ N)} & 0 \le y \le N - 1 \\
        \ket{y} & N \le y \le 2^L - 1
    \end{cases}
\end{equation*}
$$

其中 $y \in \{0, 1\}^L = [0, 2^L-1]$。这个矩阵 $U$ 有的特征状态如下：

$$
\begin{equation*}
    \ket{u_s} = \frac{1}{\sqrt{r}}\sum_{k=0}^{r-1} e^{-2\pi isk/r} \ket{x^k\ \text{mod}\ N}
\end{equation*}
$$

我们可以代入验证这一点：

$$
\begin{align*}
    U\ket{u_s}
        &= \frac{1}{\sqrt{r}} \sum_{k=0}^{r-1} e^{-2\pi isk/r} \ket{x^{k+1}\ \text{mod}\ N} \\
        &= \frac{1}{\sqrt{r}} \left[ e^{-2\pi is\cdot0/r}\ket{x\ \text{mod}\ N} + \dots + e^{-2\pi is\cdot (r-1)/r}\ket{x^r\ \text{mod}\ N} \right] \\
        &= \frac{1}{\sqrt{r}} \left[ e^{-2\pi is\cdot (r-1)/r}\ket{1\ \text{mod}\ N} + e^{-2\pi is\cdot 0/r}\ket{x\ \text{mod}\ N} + \dots + e^{-2\pi is\cdot (r-1)/r}\ket{x^{r-1}\ \text{mod}\ N} \right] \\
        &= \frac{1}{\sqrt{r}} e^{2\pi is/r} \left[ e^{-2\pi is\cdot r/r}\ket{1\ \text{mod}\ N} + e^{-2\pi is\cdot 1/r}\ket{x\ \text{mod}\ N} + \dots + e^{-2\pi is\cdot r/r}\ket{x^{r-1}\ \text{mod}\ N} \right] \\
        &= e^{2\pi is/r}\ket{u_s}
\end{align*}
$$

因此，只要我们将该状态放到相位估计算法中，就能得到与 $r$ 相关的等式，即：

$$
\begin{equation*}
    \varphi = \frac{s}{r} \implies r = \frac{s}{\varphi}
\end{equation*}
$$

为了让相位估计算法生效，我们理应找出特定的特征状态 $\ket{u_s}$；然而它本身就依赖于 $r$，这边成了鸡生蛋还是蛋生鸡的问题；因此我们需要找到这些特征状态的一个线性组合，像上节那样得到其中的某个特征值估计。在尝试解决这个问题之前，先考虑 $\ket{u_s}$ 用 $\ket{x^k\ \text{mod}\ N}$ 的表示方式：

$$
\begin{align*}
    M_s &= \begin{pmatrix}
        \ket{u_0} \\
        \dots \\
        \ket{u_{r-1}}
    \end{pmatrix} = \frac{1}{\sqrt{r}}\begin{pmatrix}
        e^{-2\pi i\cdot 0\cdot 0/r}\ket{1\ \text{mod}\ N} + \dots + e^{-2\pi i\cdot 0\cdot (r-1)/r}\ket{x^{r-1}\ \text{mod}\ N} \\
        \dots \\
        e^{-2\pi i\cdot (r-1)\cdot 0/r}\ket{1\ \text{mod}\ N} + \dots + e^{-2\pi i\cdot (r-1)\cdot (r-1)/r}\ket{x^{r-1}\ \text{mod}\ N}
    \end{pmatrix} \\
    &= \frac{1}{\sqrt{r}}\,\begin{pmatrix}
        e^{-2\pi i\cdot 0\cdot 0/r} & \dots & e^{-2\pi i\cdot 0\cdot (r-1)/r} \\
        \vdots & \ddots & \vdots \\
        e^{-2\pi i\cdot (r-1)\cdot 0/r} & \dots & e^{-2\pi i\cdot (r-1)\cdot(r-1)/r}
    \end{pmatrix}\begin{pmatrix}
        \ket{1\ \text{mod}\ N} \\
        \dots \\
        \ket{x^{r-1}\ \text{mod}\ N}
    \end{pmatrix} \\
    &= \frac{1}{\sqrt{r}}\,\begin{pmatrix}
        1 & 1 & \dots & 1 \\
        1 & e^{2\pi i(-1/r)} & \dots & e^{2\pi i[-(r-1)/r]} \\
        \vdots & \vdots & \ddots & \vdots \\
        1 & e^{2\pi i[-(r-1)/r]} & \dots & e^{2\pi i[-(r-1)^2/r]}
    \end{pmatrix}\begin{pmatrix}
        \ket{1\ \text{mod}\ N} \\
        \dots \\
        \ket{x^{r-1}\ \text{mod}\ N}
    \end{pmatrix} \\
    &= \frac{1}{\sqrt{r}}\,\begin{pmatrix}
        \omega_0^0 & \omega_0^1 & \omega_0^2 & \dots & \omega_0^{r-1} \\
        \omega_1^0 & \omega_1^1 & \omega_1^2 & \dots & \omega_1^{r-1} \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        \omega_{r-1}^0 & \omega_{r-1}^1 & \omega_{r-1}^2 & \dots & \omega_{r-1}^{r-1} 
    \end{pmatrix}\begin{pmatrix}
        \ket{1\ \text{mod}\ N} \\
        \dots \\
        \ket{x^{r-1}\ \text{mod}\ N}
    \end{pmatrix} \\
    &= U_\omega M_x
\end{align*}
$$

其中 $\omega_j$ 是 $1$ 的 $r$ 次方根且满足：

$$
\begin{equation*}
    \omega_k = \omega_1^k \qquad 
    \omega_i^j\omega_k^l = \omega_1^{ij + kl} \qquad
    \omega_i^j\omega_j^{-i} = 1
\end{equation*}
$$

这里我们直接给出结论，$U_\omega$ 是一个幺正矩阵，因此其逆是自身的共轭转置：

$$
\begin{equation*}
     U_\omega^{-1} = U_\omega^\dagger = \frac{1}{\sqrt{r}}\begin{pmatrix}
        \omega_0^0 & \omega_0^{-1} & \omega_0^{-2} & \dots & \omega_0^{-(r-1)} \\
        \omega_1^0 & \omega_1^{-1} & \omega_1^{-2} & \dots & \omega_1^{-(r-1)} \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        \omega_{r-1}^0 & \omega_{r-1}^{-1} & \omega_{r-1}^{-2} & \dots & \omega_{r-1}^{-(r-1)} 
    \end{pmatrix}
\end{equation*}
$$

事实上我们也不难验证 $ U_\omega^\dagger U_\omega = I$。因此，我们可以将 $\ket{x^k\ \text{mod}\ N}$ 表示成 $\ket{u_s}$ 的线性组合：

$$
\begin{equation*}
    \ket{x^k\ \text{mod}\ N} = \frac{1}{\sqrt{r}}\sum_{s=0}^{r-1} e^{2\pi isk/r}\ket{u_s}
\end{equation*}
$$

特别地，当 $k$ 取 $0$ 时，我们就得到了一个重要的等式：

$$
\begin{equation*}
    \frac{1}{\sqrt{r}}\sum_{s=0}^{r-1} \ket{u_s} = \ket{1}
\end{equation*}
$$

所以只需要将 $\ket{1}$ 作为初始状态传入相位估计算法即可。

此外，还有一个不太容易注意到的优化点。相位估计算法要求对幺正矩阵 $U$ 进行多次乘方操作；不过比起计算 $U^{2^j}$，我们可以直接生成这个矩阵：

$$
\begin{equation*}
    \ket{y} \mapsto \ket{x^{2^j}y\ \text{mod}\ N}
\end{equation*}
$$

这个结论基于余数乘法的性质 $x^a = (x\ \text{mod}\ N)^a\ \text{mod}\ N$。

总结下来，对于 $t=2L+1+\lceil\log(2+\frac{1}{2\epsilon})\rceil$ 的设置，我们有 $(1-\epsilon)/r$ 的概率得到准确度不小于 $2L+1$ 位的结果 $\varphi \approx s/r$。但是显然我们不能对一个估计值感到满意；这里为了计算出精准的 $r$，我们最好找到 $\varphi$ 的精确值。好消息是，由于 $\varphi$ 是一个有理数，我们可以尝试通过连分数逼近它。事实上下面这个定理为我们马上要介绍的逼近算法提供了理论基础：

> 若 $\frac{s}{r}$ 是一个有理数且：
$$
\begin{equation*}
    \left|\frac{s}{r} - \varphi\right| \le \frac{1}{2r^2}
\end{equation*}
$$
则 $\frac{s}{r}$ 可以构成 $\varphi$ 的连分数表示。其需要 $O(L^3)$ 时间构建。

对于我们的寻阶算法，由于 $s/r$ 的精确度不小于 $2L + 1$，因此：

$$
\begin{equation*}
    \left|\frac{s}{r} - \varphi\right| \le \frac{1}{2^{2L+1}} < \frac{1}{N^2} \le \frac{1}{2r^2}
\end{equation*}
$$

#### 连分数算法

连分数是通过一系列整数 $a_k$ 表示任意实数的记号，如：

$$
\begin{equation*}
    x = [a_0; a_1, \dots, a_M] = a_0 + \frac{1}{a_1 + \frac{1}{a_2 + \frac{1}{\dots + \frac{1}{a_m}}}}
\end{equation*}
$$

此时我们称 $a_0, \dots, a_m$ 在 $m$ 阶收敛于 $x$。**连分数算法（Continued Fraction Algorithm）** 给出了一种确定任意实数的连分数形式的方法。这里我们以一个有理数为例：

$$
\begin{align*}
    \frac{23}{12} 
        &= 1 + \frac{11}{12} \\
        &= 1 + \frac{1}{1 + \frac{1}{11}} \\
        &= [1; 1, 11]
\end{align*}
$$

事实上，对于所有的有理数 $x$，我们总能找到有限的数列 $a_0, a_1, \dots, a_m$ 使得 $[a_0; a_1, \dots, a_m] = x$。

#### 回顾寻阶算法

寻阶算法只经过一次测量有可能找不到想要的结果 $r$：首先你根据 $t$ 的选取有最多 $\epsilon$ 的概率得到不精准的 $\varphi$；此外，也有一定概率得到不和 $s$ 互素的 $r$。根据素数的分布规律，选取随机的 $s$ 使其是复数的概率至少是 $1/(2\log{r}) > 1/(2\log{N})$。因此我们重复进行 $2\log{N}$ 次测量后大概率至少得到一个满足条件的 $r$。

### 舒尔算法

经过了前面几节的铺垫，我们终于迎来一个重量级的 **素因数分解（Factoring）** 算法，**舒尔算法（Shor Algorithm）**。它基于下面这个定理（实际上它非常显然）：

> 对于宽度为 $L$ 比特的合数 $N$，如果 $x^2=1\ (\text{mod} N)$ 有非平凡解 $2 \le x \le N-2$，则 $\text{gcd}(x+1, N)$ 和 $\text{gcd}(x-1, N)$ 中至少存在一个 $N$ 的非平凡因数；这个过程可以通过 $O(L^3)$ 次运算得到。 

舒尔算法的步骤如下所示：

- 若 $N$ 是偶数，返回因数 $2$。
- 对于 $a \ge 1$ 且 $b \ge 2$ 的整数判断是否 $N = a^b$，若是则返回 $a$。
- 随机选取 $1 \le x \le N - 1$，若 $\text{gcd}(x, N) > 1$ 则返回 $\text{gcd}(x, N)$。
- 用寻阶算法寻找 $x$ 模 $N$ 的阶 $r$。
- 若 $r$ 是偶数且 $x^{r/2} + 1 \ne 0\ (\text{mod} N)$，计算 $\text{gcd}(x^{r/2}-1, N)$ 和 $\text{gcd}(x^{r/2}+1, N)$，若任一个是 $N$ 的非平凡因数则返回之。其它情况下算法失败。